# **安装依赖库**

## **作者简介**

#### 作者：**⚡小闪电⚡**

#### B站主页
- [小闪电的B站主页](https://space.bilibili.com/122699831?spm_id_from=333.1007.0.0)

#### 交流群
欢迎加入AIDD交流群！  
加我微信（微信号: `xxxFLASHxxx`），邀请你进群。

## **安装依赖库**

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install rdkit
!pip install networkx pandas
%matplotlib inline
import matplotlib.pyplot as plt
from torch_geometric.datasets import KarateClub
dataset = KarateClub()
print(f'Dataset: {dataset}:')

# **小分子描述符、指纹及MLP**

## **数据分析可视化**

### 先看看数据集的甜度分布

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ==========================================
# 1. 加载数据
# ==========================================
csv_path = '/content/SweetpredDB.csv'
df = pd.read_csv(csv_path)

print(f"数据加载成功！共包含 {len(df)} 个样本。")
print("前5行数据预览：")
print(df[['Smiles', 'logSw']].head())

# ==========================================
# 2. 可视化甜度值 (logSw) 分布
# ==========================================

# 设置高清 DPI
plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600
sns.set_style("whitegrid") # 设置背景网格

# 定义颜色
color_blue = (75 / 255, 116 / 255, 178 / 255)
color_red = (219 / 255, 49 / 255, 36 / 255)

plt.figure(figsize=(10, 6),dpi=100)

# 绘制直方图 + KDE 密度曲线
sns.histplot(
    data=df,
    x='logSw',
    kde=True,
    color=color_blue,
    bins=30,
    edgecolor='white',
    alpha=0.7,
    line_kws={'linewidth': 2}
)

# 添加阈值线 (x=3)
plt.axvline(x=3, color=color_red, linestyle='--', linewidth=2, label='Threshold (3.0)')

# 添加平均值线
mean_val = df['logSw'].mean()
plt.axvline(x=mean_val, color='gray', linestyle=':', linewidth=2, label=f'Mean ({mean_val:.2f})')

# 添加文字标注
plt.text(3.1, plt.gca().get_ylim()[1]*0.85, 'High Sweetness (>3)',
         color=color_red, fontsize=11, fontweight='bold')
plt.text(2.9, plt.gca().get_ylim()[1]*0.85, 'Low Sweetness (<=3)',
         color=color_blue, fontsize=11, fontweight='bold', ha='right')

# 图表设置
plt.title('Distribution of Sweetness Values (logSw)', fontsize=14, fontweight='bold')
plt.xlabel('logSw Value', fontsize=12)
plt.ylabel('Count / Density', fontsize=12)
plt.legend()

plt.tight_layout()
plt.show()

### 计算分子描述符和分子指纹

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from rdkit import Chem
from rdkit import rdBase
from rdkit.Chem import Descriptors, AllChem, Draw
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm
import warnings

# ==========================================
# 配置与设置
# ==========================================

# 1. 屏蔽 Python 警告
warnings.filterwarnings('ignore')

# 2. 屏蔽 RDKit 警告
rdBase.DisableLog('rdApp.warning')

# 3. 屏蔽 Matplotlib 字体查找警告 (解决 SimHei not found 刷屏)
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

# 4. 启用 Colab 的交互式表格显示
try:
    from google.colab import data_table
    data_table.enable_dataframe_formatter()
except ImportError:
    print("注意: 当前环境非 Google Colab，无法使用交互式表格功能。")

# 设置绘图风格 (移除 SimHei 设置以避免警告，使用默认字体)
plt.style.use('seaborn-v0_8-whitegrid')
# plt.rcParams['font.sans-serif'] = ['SimHei'] # 注释掉这行，避免报错
plt.rcParams['axes.unicode_minus'] = False

# ==========================================
# 1. 数据加载与特征计算函数
# ==========================================

def get_mol_descriptors(mol):
    """计算常用的分子描述符"""
    return {
        'MolWt': Descriptors.MolWt(mol),           # 分子量
        'LogP': Descriptors.MolLogP(mol),          # 脂水分配系数
        'NumHDonors': Descriptors.NumHDonors(mol), # 氢键供体数
        'NumHAcceptors': Descriptors.NumHAcceptors(mol), # 氢键受体数
        'TPSA': Descriptors.TPSA(mol),             # 拓扑极性表面积
        'RotatableBonds': Descriptors.NumRotatableBonds(mol) # 可旋转键数
    }

def get_morgan_fingerprint(mol, n_bits=2048):
    """计算Morgan分子指纹 (ECFP4)"""
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits)
    return np.array(fp)

# 加载数据
csv_path = '/content/SweetpredDB.csv'
df = pd.read_csv(csv_path)

# 清洗数据：移除无效分子
valid_data = []

print("正在计算分子特征...")

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing Molecules"):
    mol = Chem.MolFromSmiles(row['Smiles'])
    if mol is not None:
        # 计算描述符
        desc = get_mol_descriptors(mol)
        # 计算指纹
        fp = get_morgan_fingerprint(mol)

        entry = {
            'Smiles': row['Smiles'],
            'logSw': row['logSw'],
            'Fingerprint': fp, # 这是一个长度2048的数组
            'Mol': mol
        }
        entry.update(desc)
        valid_data.append(entry)

df_processed = pd.DataFrame(valid_data)
print(f"\n数据处理完成。原始数量: {len(df)}, 有效数量: {len(df_processed)}")

In [ ]:
# ==========================================
# 2. 单个分子可视化与特征展示
# ==========================================
print("\n" + "="*40)
print("分子展示")
print("="*40)

if not df_processed.empty:
    sample_row = df_processed.sample(1).iloc[0]

    # 绘制分子结构
    img = Draw.MolToImage(sample_row['Mol'], size=(300, 300))
    plt.figure(figsize=(4, 4),dpi=100)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"LogSw: {sample_row['logSw']:.2f}")
    plt.show()

    # 打印文本信息
    print(f"SMILES: {sample_row['Smiles']}")
    print("-" * 20)
    print("分子描述符 (Descriptors):")
    for k in ['MolWt', 'LogP', 'NumHDonors', 'NumHAcceptors', 'TPSA', 'RotatableBonds']:
        print(f"{k}: {sample_row[k]:.2f}")
    print("-" * 20)

In [ ]:
# ==========================================
# 3. 使用 Colab 表格展示数据 (随机抽样)
# ==========================================
from google.colab import data_table

print("\n" + "="*40)
print("随机抽取 10 个分子的指纹预览")
print("="*40)

if not df_processed.empty:
    # 1. 随机抽取 10 行 (如果总数不足10则抽取全部)
    # random_state=42 保证每次抽到的都是同样的一批，想完全随机可以去掉它
    sample_size = min(20, len(df_processed))
    sampled_df = df_processed.sample(n=sample_size, random_state=42)

    # 2. 展开指纹 (关键：指定 index=sampled_df.index 以便与原始数据对齐)
    fp_expanded = pd.DataFrame(sampled_df['Fingerprint'].tolist(), index=sampled_df.index)
    fp_expanded.columns = [f'FP_{i}' for i in range(fp_expanded.shape[1])]

    # 3. 合并 Smiles 和 指纹列
    display_df = pd.concat([sampled_df[['Smiles']], fp_expanded], axis=1)

    # 4. 设置 Colab 表格的最大列数限制
    data_table.DataTable.max_columns = 2100

    # 5. 显示表格
    display(display_df)

### 瞅一瞅各种分子描述符和甜度关系大不大

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import numpy as np

# ==========================================
# 2. 可视化配置与数据准备
# ==========================================

# 设置全局 DPI 为 600，保证所有图表输出的高清质量
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 200

# 定义颜色 (按你要求的 RGB 归一化)
color_blue = (75 / 255, 116 / 255, 178 / 255)  # 低甜
color_red = (219 / 255, 49 / 255, 36 / 255)    # 高甜
custom_palette = [color_blue, color_red]

# 定义需要分析的描述符列名
descriptor_cols = ['MolWt', 'LogP', 'NumHDonors', 'NumHAcceptors', 'TPSA', 'RotatableBonds']

# 划分甜度类别：以 3 为界限
# 这里假设 logSw 是你的甜度指标
df_processed['Sweetness_Class'] = df_processed['logSw'].apply(
    lambda x: 'High Sweetness (>3)' if x > 3 else 'Low Sweetness (<=3)'
)

# 确保分类顺序 (Low -> Blue, High -> Red)
class_order = ['Low Sweetness (<=3)', 'High Sweetness (>3)']

# ==========================================
# 3. 可视化 A：分子描述符与甜度的线性关系 (散点 + 拟合线)
# ==========================================
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, col in enumerate(descriptor_cols):
    # 使用 regplot 绘制散点和线性回归拟合线
    sns.regplot(
        x=col,
        y='logSw',
        data=df_processed,
        ax=axes[i],
        scatter_kws={'alpha': 0.5, 'color': 'gray', 's': 15}, # 散点样式
        line_kws={'color': 'black', 'linewidth': 1.5}         # 拟合线样式
    )
    axes[i].set_title(f'{col} vs logSw', fontsize=12, fontweight='bold')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('logSw (Sweetness)')

plt.suptitle('Linear Relationship: Molecular Descriptors vs Sweetness', fontsize=16, y=1.02)
plt.tight_layout()
# 如果需要保存图片，可以使用 plt.savefig('scatter_plots.png')
plt.show()

# ==========================================
# 4. 可视化 B：高甜 vs 低甜 的描述符分布 (箱线图)
# ==========================================
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, col in enumerate(descriptor_cols):
    # 箱线图
    sns.boxplot(
        x='Sweetness_Class',
        y=col,
        data=df_processed,
        order=class_order,
        palette=custom_palette,
        ax=axes[i],
        width=0.5,
        showfliers=False # 不显示异常值点，避免太乱，由 stripplot 展示
    )
    # 抖动散点图 (展示真实数据分布)
    sns.stripplot(
        x='Sweetness_Class',
        y=col,
        data=df_processed,
        order=class_order,
        color=".25",
        alpha=0.3,
        size=3,
        ax=axes[i]
    )

    axes[i].set_title(f'{col} Distribution', fontsize=12, fontweight='bold')
    axes[i].set_xlabel('')
    axes[i].set_ylabel(col)

plt.suptitle('Distribution Comparison: High vs Low Sweetness', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# ==========================================
# 1. 相关性热力图 (ClusterMap)
# ==========================================
print(">>> [1/2] 正在绘制相关性聚类热力图 (ClusterMap)...")

# 确保 descriptor_cols 已定义
if 'descriptor_cols' not in locals():
    descriptor_cols = ['MolWt', 'LogP', 'NumHDonors', 'NumHAcceptors', 'TPSA', 'RotatableBonds']

# 选择用于计算相关性的数值列 (描述符 + 甜度)
cols_to_plot = descriptor_cols + ['logSw']

# 计算相关系数矩阵
corr_matrix = df_processed[cols_to_plot].corr()

# 绘制聚类热力图
g = sns.clustermap(
    corr_matrix,
    annot=True,       # 显示数值
    fmt=".2f",        # 数值保留2位小数
    cmap='coolwarm',  # 红蓝配色
    center=0,         # 颜色中心设为0
    figsize=(6, 6),
    linewidths=.75,   # 格子边框宽度
    cbar_kws={"shrink": .5},
    dendrogram_ratio=(.15, .15)
)

plt.suptitle('Molecular Descriptors Correlation ClusterMap', y=1.02, fontsize=18)
plt.show()

In [ ]:
# ==========================================
# 0. 安装依赖 (如果尚未安装)
# ==========================================
!pip install pyvis

import networkx as nx
from pyvis.network import Network
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Draw
import pandas as pd
import numpy as np
import base64
from io import BytesIO
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from IPython.display import display, HTML

# ==========================================
# 1. 辅助函数定义
# ==========================================

def mol_to_base64_html(mol):
    """将 RDKit 分子对象转换为简单的 HTML img 标签"""
    if mol is None:
        return ""
    img = Draw.MolToImage(mol, size=(200, 200))
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    # 返回 img 标签
    return f'<img src="data:image/png;base64,{img_str}" width="200" height="200">'

def get_color_hex(value, vmin, vmax, cmap_name='coolwarm'):
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    cmap = cm.get_cmap(cmap_name)
    rgba = cmap(norm(value))
    return mcolors.to_hex(rgba)

# ==========================================
# 2. 数据准备：强制平衡采样
# ==========================================
print(">>> [1/3] 正在准备平衡的高甜/低甜分子数据...")

# 1. 分离高甜和低甜数据
# 假设 df_processed 已经存在
df_high = df_processed[df_processed['logSw'] > 3]
df_low = df_processed[df_processed['logSw'] <= 3]

# 2. 确定每类采样数量 (取两者最小值的 50，防止图太大卡顿)
n_samples = min(len(df_high), len(df_low), 50)
print(f"每类采样数量: {n_samples} (总计 {n_samples*2})")

# 3. 平衡采样
subset_high = df_high.sample(n=n_samples, random_state=42)
subset_low = df_low.sample(n=n_samples, random_state=42)

# 4. 合并并打乱
subset = pd.concat([subset_high, subset_low]).sample(frac=1, random_state=42).reset_index(drop=True)

# 准备分子指纹
mols = [Chem.MolFromSmiles(s) for s in subset['Smiles']]
fps = [AllChem.GetMorganFingerprintAsBitVect(x, 2, nBits=1024) for x in mols]

# 预先计算颜色范围
min_sw = subset['logSw'].min()
max_sw = subset['logSw'].max()

# ==========================================
# 3. 构建 PyVis 网络
# ==========================================
print(f">>> [2/3] 正在构建交互式网络 (阈值 Tanimoto > 0.5)...")

# 关键设置：cdn_resources='in_line' 确保所有脚本内嵌，不需要联网加载 CDN
net = Network(height="750px", width="100%", notebook=True, cdn_resources='in_line', bgcolor="#ffffff", font_color="black")

n = len(subset)
threshold = 0.5

# --- 添加节点 ---
for i in range(n):
    mol = mols[i]
    val = subset.loc[i, 'logSw']
    smiles = subset.loc[i, 'Smiles']

    img_html = mol_to_base64_html(mol)

    # 构造 Tooltip (HTML格式)
    # 包含 ID, 甜度, SMILES 和 分子结构图
    title_html = (
        f"<b>ID:</b> {i}<br>"
        f"<b>Sweetness:</b> {val:.3f}<br>"
        f"<b>SMILES:</b> {smiles[:15]}...<br>"
        f"{img_html}"
    )

    node_color = get_color_hex(val, min_sw, max_sw)

    # 添加节点
    net.add_node(i, label=str(i), title=title_html, color=node_color, size=20, borderWidth=1)

# --- 添加边 ---
for i in range(n):
    for j in range(i + 1, n):
        sim = DataStructs.TanimotoSimilarity(fps[i], fps[j])
        if sim > threshold:
            # 边越粗代表越相似
            width = (sim - threshold) * 5
            net.add_edge(i, j, value=sim, title=f"Similarity: {sim:.2f}", color='#cccccc', width=width)

# ==========================================
# 4. 在 Colab 中直接渲染 (不下载)
# ==========================================
print(">>> [3/3] 正在渲染可视化...")

# 添加物理控制面板
net.show_buttons(filter_=['physics'])

# 保存为 HTML 文件
html_file_name = "balanced_sweetness_network.html"
net.save_graph(html_file_name)

# 核心魔法：读取 HTML 内容并直接显示在 Output 区域
print("✅ 交互式图表已生成，请直接在下方查看：")
display(HTML(filename=html_file_name))

### T-SNE降维可视化一下高甜/低甜分子的分子描述符和分子指纹

In [ ]:

# ==========================================
# 5. 可视化 C：t-SNE 降维可视化
# ==========================================

print("正在计算 t-SNE，请稍候...")

# --- 5.1 基于分子描述符 (Descriptors) 的 t-SNE ---

# 提取描述符数据并标准化
X_desc = df_processed[descriptor_cols].values
scaler = StandardScaler()
X_desc_scaled = scaler.fit_transform(X_desc)

# 计算 t-SNE
tsne_desc = TSNE(n_components=2, random_state=42, perplexity=30, init='pca', learning_rate='auto')
X_embedded_desc = tsne_desc.fit_transform(X_desc_scaled)

# 将结果存入 DataFrame
df_processed['tsne_desc_x'] = X_embedded_desc[:, 0]
df_processed['tsne_desc_y'] = X_embedded_desc[:, 1]


# --- 5.2 基于分子指纹 (Fingerprints) 的 t-SNE ---

# 提取指纹数据 (将 Series 中的 array 堆叠成矩阵)
X_fp = np.stack(df_processed['Fingerprint'].values)

# 计算 t-SNE
tsne_fp = TSNE(n_components=2, random_state=42, perplexity=30, init='pca', learning_rate='auto')
X_embedded_fp = tsne_fp.fit_transform(X_fp)

df_processed['tsne_fp_x'] = X_embedded_fp[:, 0]
df_processed['tsne_fp_y'] = X_embedded_fp[:, 1]


# --- 5.3 绘制 t-SNE 结果 ---

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# 绘图函数封装
def plot_tsne(ax, x_col, y_col, title):
    sns.scatterplot(
        x=x_col, y=y_col,
        hue='Sweetness_Class',
        hue_order=class_order,
        palette=custom_palette,
        data=df_processed,
        alpha=0.8,
        s=60,
        ax=ax
    )
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.legend(title='Sweetness Class', loc='upper right')
    ax.set_xlabel('t-SNE Dim 1')
    ax.set_ylabel('t-SNE Dim 2')

# 画图
plot_tsne(axes[0], 'tsne_desc_x', 'tsne_desc_y', 't-SNE: Molecular Descriptors')
plot_tsne(axes[1], 'tsne_fp_x', 'tsne_fp_y', 't-SNE: Morgan Fingerprints (ECFP4)')

plt.suptitle('Manifold Learning Visualization (t-SNE)', fontsize=16, y=0.95)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from plotly.subplots import make_subplots

# ==========================================
# 5. 可视化 C：3D t-SNE 降维可视化 (交互式)
# ==========================================

print("正在计算 3D t-SNE，计算量稍大，请稍候...")

# -------------------------------------------------------
# 1. 数据准备与计算 (n_components=3)
# -------------------------------------------------------

# --- A. 基于分子描述符 (Descriptors) ---
# 提取并标准化
X_desc = df_processed[descriptor_cols].values
scaler = StandardScaler()
X_desc_scaled = scaler.fit_transform(X_desc)

# 计算 3D t-SNE
tsne_desc = TSNE(n_components=3, random_state=42, perplexity=30, init='pca', learning_rate='auto')
X_embedded_desc = tsne_desc.fit_transform(X_desc_scaled)

# 存入 DataFrame
df_processed['tsne_desc_x'] = X_embedded_desc[:, 0]
df_processed['tsne_desc_y'] = X_embedded_desc[:, 1]
df_processed['tsne_desc_z'] = X_embedded_desc[:, 2] # 新增 Z 轴


# --- B. 基于分子指纹 (Fingerprints) ---
# 提取指纹矩阵
X_fp = np.stack(df_processed['Fingerprint'].values)

# 计算 3D t-SNE
tsne_fp = TSNE(n_components=3, random_state=42, perplexity=30, init='pca', learning_rate='auto')
X_embedded_fp = tsne_fp.fit_transform(X_fp)

# 存入 DataFrame
df_processed['tsne_fp_x'] = X_embedded_fp[:, 0]
df_processed['tsne_fp_y'] = X_embedded_fp[:, 1]
df_processed['tsne_fp_z'] = X_embedded_fp[:, 2] # 新增 Z 轴


# -------------------------------------------------------
# 2. 定义绘图函数 (使用 Plotly Express)
# -------------------------------------------------------

# 定义颜色映射 (Hex格式，对应之前的红蓝配色)
color_map = {
    'Low Sweetness (<=3)': '#4B74B2',  # 蓝色
    'High Sweetness (>3)': '#DB3124'   # 红色
}

def plot_3d_tsne(x, y, z, title, filename_suffix):
    """
    绘制单个 3D 散点图
    """
    fig = px.scatter_3d(
        df_processed,
        x=x, y=y, z=z,
        color='Sweetness_Class',
        color_discrete_map=color_map,

        # 悬停显示的信息 (非常有用！)
        hover_data={
            'Smiles': True,
            'logSw': True,
            x: False, y: False, z: False # 隐藏坐标值，只看分子信息
        },
        title=title,
        opacity=0.7,
        size_max=8
    )

    # 优化点的大小和标记
    fig.update_traces(marker=dict(size=4, line=dict(width=0)))

    # 设置布局
    fig.update_layout(
        margin=dict(l=0, r=0, b=0, t=40),
        scene=dict(
            xaxis_title='t-SNE Dim 1',
            yaxis_title='t-SNE Dim 2',
            zaxis_title='t-SNE Dim 3'
        ),
        legend_title_text='Sweetness Class'
    )

    fig.show()

# -------------------------------------------------------
# 3. 绘制并展示图表
# -------------------------------------------------------

print("\n>>> 交互式 3D 图表 1: 基于分子描述符")
plot_3d_tsne('tsne_desc_x', 'tsne_desc_y', 'tsne_desc_z',
             '3D t-SNE: Molecular Descriptors', 'desc')

print("\n>>> 交互式 3D 图表 2: 基于分子指纹")
plot_3d_tsne('tsne_fp_x', 'tsne_fp_y', 'tsne_fp_z',
             '3D t-SNE: Morgan Fingerprints', 'fp')

### 训练传统机器学习模型预测分子甜度

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# ==========================================
# 0. 数据准备
# ==========================================
SEED = 42

# A组特征：分子描述符 (标准化)
descriptor_cols = ['MolWt', 'LogP', 'NumHDonors', 'NumHAcceptors', 'TPSA', 'RotatableBonds']
X_desc = df_processed[descriptor_cols].values
scaler = StandardScaler()
X_desc_scaled = scaler.fit_transform(X_desc)

# B组特征：Morgan指纹
X_fp = np.stack(df_processed['Fingerprint'].values).astype(float)

# 目标值
y = df_processed['logSw'].values

# 同时切分两组特征，确保 y_test 一致，方便公平对比
X_desc_train, X_desc_test, X_fp_train, X_fp_test, y_train, y_test = train_test_split(
    X_desc_scaled, X_fp, y, test_size=0.2, random_state=SEED
)

# ==========================================
# 1. 定义8个模型 (4种算法 x 2种特征)
# ==========================================
# 为了方便，我们将训练集和测试集封装在元组中
# 格式: "模型名": (模型对象, 训练数据, 测试数据)

models = {
    # --- 1. 线性模型 ---
    "Linear Reg (Descriptors)": (
        LinearRegression(),
        X_desc_train, X_desc_test
    ),
    "Linear Ridge (Fingerprints)": (
        Ridge(alpha=1.0), # 指纹维度高，使用Ridge防止过拟合
        X_fp_train, X_fp_test
    ),

    # --- 2. 随机森林 (集成学习-Bagging) ---
    "Random Forest (Descriptors)": (
        RandomForestRegressor(n_estimators=100, random_state=SEED, n_jobs=-1),
        X_desc_train, X_desc_test
    ),
    "Random Forest (Fingerprints)": (
        RandomForestRegressor(n_estimators=100, random_state=SEED, n_jobs=-1),
        X_fp_train, X_fp_test
    ),

    # --- 3. SVM (支持向量机) ---
    "SVR (Descriptors)": (
        SVR(kernel='rbf', C=1.0, epsilon=0.1), # RBF核处理非线性
        X_desc_train, X_desc_test
    ),
    "SVR (Fingerprints)": (
        SVR(kernel='rbf', C=1.0, epsilon=0.1),
        X_fp_train, X_fp_test
    ),

    # --- 4. XGBoost (集成学习-Boosting) ---
    "XGBoost (Descriptors)": (
        XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=SEED, n_jobs=-1),
        X_desc_train, X_desc_test
    ),
    "XGBoost (Fingerprints)": (
        XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=SEED, n_jobs=-1),
        X_fp_train, X_fp_test
    )
}

# ==========================================
# 2. 训练与评估
# ==========================================
results = []
predictions = {}

print(f"{'Model Name':<30} | {'RMSE':<10} | {'R2 Score':<10}")
print("-" * 55)

for name, (model, X_tr, X_te) in models.items():
    # 训练
    model.fit(X_tr, y_train)
    # 预测
    y_pred = model.predict(X_te)
    predictions[name] = y_pred

    # 评估
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    results.append({"Model": name, "RMSE": rmse, "R2": r2})
    print(f"{name:<30} | {rmse:<10.4f} | {r2:<10.4f}")

print("-" * 55)

# ==========================================
# 3. 可视化对比 (2行4列)
# ==========================================
# 设置布局：第一行是基于描述符的模型，第二行是基于指纹的模型
fig, axes = plt.subplots(2, 4, figsize=(24, 10), dpi=600)
# 扁平化轴数组方便遍历，但我们需要按特定顺序填入
# 我们手动指定顺序以保持对齐：
# 列顺序: Linear, RF, SVR, XGB

# 定义绘图顺序对应的键名
plot_order = [
    ["Linear Reg (Descriptors)", "Random Forest (Descriptors)", "SVR (Descriptors)", "XGBoost (Descriptors)"],
    ["Linear Ridge (Fingerprints)", "Random Forest (Fingerprints)", "SVR (Fingerprints)", "XGBoost (Fingerprints)"]
]

color_scatter = (75/255, 116/255, 178/255)
color_line = (219/255, 49/255, 36/255)

for row in range(2):
    for col in range(4):
        ax = axes[row, col]
        model_name = plot_order[row][col]
        y_pred = predictions[model_name]

        # 获取该模型的分数
        res = next(item for item in results if item["Model"] == model_name)

        # 散点图
        ax.scatter(y_test, y_pred, alpha=0.8, color=color_scatter, s=30)

        # 完美预测线
        min_val = min(y_test.min(), y_pred.min())
        max_val = max(y_test.max(), y_pred.max())
        ax.plot([min_val, max_val], [min_val, max_val], '--', color=color_line, lw=2)

        # 标注
        ax.text(0.05, 0.95, f'$R^2 = {res["R2"]:.3f}$\n$RMSE = {res["RMSE"]:.3f}$',
                 transform=ax.transAxes, verticalalignment='top',
                 bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

        ax.set_title(model_name, fontsize=11, fontweight='bold')
        ax.set_xlabel('True')
        ax.set_ylabel('Pred')
        ax.grid(True, alpha=0.3)

plt.suptitle("Benchmark: Descriptors (Row 1) vs Fingerprints (Row 2)", fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

# ==========================================
# 4. 自动生成分析报告
# ==========================================
# 找出R2最高的模型
best_model = max(results, key=lambda x: x['R2'])

print("\n" + "="*20 + " 深度分析 " + "="*20)
print(f"🏆 最佳传统模型: {best_model['Model']} (R2={best_model['R2']:.4f})")
print("\n🔍 核心观察点：")
print("1. 特征对比：通常情况下，基于指纹(Fingerprints)的模型上限更高，因为包含了更多结构信息；")
print("   但描述符(Descriptors)在简单模型上可能更稳定，且计算极快。")
print("2. 线性 vs 非线性：观察 Linear Reg 是否显著弱于 RF/XGB/SVR？")
print("   如果是，说明分子甜度具有强烈的非线性特征（构效关系复杂）。")
print("3. XGBoost vs RF：这两个都是集成学习强者，XGBoost通常稍胜一筹，但也更难调参。")
print("4. GNN的挑战：GNN 需要击败这里表现最好的模型（例如 XGBoost+Fingerprints）才能证明其端到端学习的价值。")
print("="*50)

### 训练一个MLP模型预测分子甜度

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt

# ==========================================
# 0. 配置与设备选择
# ==========================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 设置绘图参数
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
color_blue = (75 / 255, 116 / 255, 178 / 255)
color_red = (219 / 255, 49 / 255, 36 / 255)

# ==========================================
# 1. 数据准备
# ==========================================
# (假设 df_processed, descriptor_cols 已经在环境中定义)

# --- A. 准备特征 ---
# 1. 描述符
X_desc_raw = df_processed[descriptor_cols].values
scaler_desc = StandardScaler()
X_desc_scaled = scaler_desc.fit_transform(X_desc_raw)

# 2. 指纹
X_fp_raw = np.stack(df_processed['Fingerprint'].values)
X_fp_float = X_fp_raw.astype(float)

# --- B. 目标变量 ---
y_val = df_processed['logSw'].values.reshape(-1, 1)

# --- C. 数据加载器 ---
def create_dataloaders(X, y, batch_size=32):
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
    X_val, X_test, y_val_set, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

    tensor_X_train = torch.FloatTensor(X_train).to(device)
    tensor_y_train = torch.FloatTensor(y_train).to(device)
    tensor_X_val = torch.FloatTensor(X_val).to(device)
    tensor_y_val = torch.FloatTensor(y_val_set).to(device)
    tensor_X_test = torch.FloatTensor(X_test).to(device)
    tensor_y_test = torch.FloatTensor(y_test).to(device)

    train_loader = DataLoader(TensorDataset(tensor_X_train, tensor_y_train), batch_size=batch_size, shuffle=True)
    return train_loader, tensor_X_val, tensor_y_val, tensor_X_test, tensor_y_test

print("准备数据...")
data_desc = create_dataloaders(X_desc_scaled, y_val)
data_fp = create_dataloaders(X_fp_float, y_val)

# ==========================================
# 2. 定义更深的 MLP 模型 (显式定义层结构)
# ==========================================

class SweetNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super(SweetNet, self).__init__()
        self.net = nn.Sequential(
            # --- 第1层隐藏层 (Input -> Hidden) ---
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),

            # --- 第2层隐藏层 (Hidden -> Hidden/2) ---
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3),

            # --- 第3层隐藏层 (Hidden/2 -> Hidden/4) ---
            # 这是新增的一层，让网络变深
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Dropout(0.2),

            # --- 输出层 (Hidden/4 -> Output) ---
            nn.Linear(hidden_dim // 4, 1)
        )

    def forward(self, x):
        return self.net(x)

# ==========================================
# 3. 训练函数
# ==========================================
def train_model(model, data_pack, epochs=100, lr=0.001, name="Model"):
    train_loader, X_val, y_val, X_test, y_test = data_pack
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses = []
    val_losses = []

    model.train()
    print(f"开始训练 {name} ...")

    for epoch in range(epochs):
        batch_losses = []
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            batch_losses.append(loss.item())

        avg_train_loss = np.mean(batch_losses)
        train_losses.append(avg_train_loss)

        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val).item()
            val_losses.append(val_loss)
        model.train()

        if (epoch+1) % 1 == 0:
            print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f}")

    model.eval()
    with torch.no_grad():
        test_preds = model(X_test).cpu().numpy()
        y_test_np = y_test.cpu().numpy()

    metrics = {
        'rmse': np.sqrt(mean_squared_error(y_test_np, test_preds)),
        'r2': r2_score(y_test_np, test_preds)
    }
    return train_losses, val_losses, y_test_np, test_preds, metrics

# ==========================================
# 4. 执行训练
# ==========================================

# --- 模型 A: 描述符模型 (输入维度较小，隐藏层给大一点) ---
model_desc = SweetNet(input_dim=6, hidden_dim=128).to(device)
print(f"\n模型结构 (Descriptors):\n{model_desc}")
hist_desc = train_model(model_desc, data_desc, epochs=150, lr=0.005, name="Descriptors MLP")

# --- 模型 B: 指纹模型 (输入维度大，隐藏层给更大) ---
model_fp = SweetNet(input_dim=2048, hidden_dim=512).to(device)
print(f"\n模型结构 (Fingerprints):\n{model_fp}")
hist_fp = train_model(model_fp, data_fp, epochs=150, lr=0.001, name="Fingerprints MLP")

# ==========================================
# 5. 可视化结果
# ==========================================
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

def plot_results(ax_loss, ax_pred, hist, title_prefix):
    # Loss
    ax_loss.plot(hist[0], label='Train', color=color_blue)
    ax_loss.plot(hist[1], label='Val', color=color_red, linestyle='--')
    ax_loss.set_title(f'{title_prefix} Loss')
    ax_loss.legend()
    ax_loss.grid(True, alpha=0.3)

    # Pred
    y_true, y_pred, metrics = hist[2], hist[3], hist[4]
    ax_pred.scatter(y_true, y_pred, alpha=0.8, color=color_blue, s=30)
    min_v, max_v = min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max())
    ax_pred.plot([min_v, max_v], [min_v, max_v], '--', color=color_red, lw=2)
    ax_pred.text(0.05, 0.95, f'R2={metrics["r2"]:.3f}\nRMSE={metrics["rmse"]:.3f}',
                 transform=ax_pred.transAxes, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    ax_pred.set_title(f'{title_prefix} Pred')
    ax_pred.grid(True, alpha=0.3)

plot_results(axes[0,0], axes[0,1], hist_desc, "Descriptors")
plot_results(axes[1,0], axes[1,1], hist_fp, "Fingerprints")

plt.tight_layout()
plt.show()

# **小分子任务——基于GNN的化合物甜度预测-图级别任务**

## **1-制作Graph数据集**

In [ ]:
import pandas as pd
from rdkit import Chem
from torch_geometric.data import Data
import torch
import os
from tqdm import tqdm
from scipy.sparse import coo_matrix
import numpy as np

# 辅助函数
def one_hot_encoding(value, choices):
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1
    return encoding

# 特征提取器类（修复了边特征错位问题）
class MoleculeFeaturizer(object):
    def __init__(self, bond_features=True):
        self.bond_features = bond_features

    def _atom_featurizer(self, atom):
        atomic_numer = [1, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 28, 29,
                        30, 31, 32, 33, 34, 35, 36, 37, 38, 46, 47, 48, 49, 50, 51, 52, 53]
        atom_type_value = atom.GetAtomicNum()
        atom_type_features = one_hot_encoding(atom_type_value, atomic_numer)

        degree_choices = list(range(5))
        degree = atom.GetTotalDegree()
        degree_features = one_hot_encoding(degree, degree_choices)

        hybridization_choices = list(range(len(Chem.HybridizationType.names) - 1))
        hybridization = int(atom.GetHybridization())
        hybrid_features = one_hot_encoding(hybridization, hybridization_choices)

        chiral_tag_choices = list(range(len(Chem.ChiralType.names) - 1))
        chiral_tag = atom.GetChiralTag()
        chiral_tag_features = one_hot_encoding(chiral_tag, chiral_tag_choices)

        num_Hs_choices = list(range(5))
        num_Hs = atom.GetTotalNumHs()
        num_Hs_features = one_hot_encoding(num_Hs, num_Hs_choices)

        aromatic_features = [1 if atom.GetIsAromatic() else 0]

        return atom_type_features + degree_features + hybrid_features + chiral_tag_features + num_Hs_features + aromatic_features

    def _bond_featurizer(self, bond):
        bond_type = bond.GetBondType()
        bond_type_one_hot_encoding = [
            int(bond_type == Chem.rdchem.BondType.SINGLE),
            int(bond_type == Chem.rdchem.BondType.DOUBLE),
            int(bond_type == Chem.rdchem.BondType.TRIPLE),
            int(bond_type == Chem.rdchem.BondType.AROMATIC)
        ]
        addition_bond_features = [
            int(bond.GetIsConjugated()),
            int(bond.IsInRing())
        ]
        return bond_type_one_hot_encoding + addition_bond_features

    def __call__(self, mol):
        self.feature_dict = {}

        # 原子特征
        atom_features = [self._atom_featurizer(atom) for atom in mol.GetAtoms()]
        self.feature_dict['x'] = atom_features

        # 边索引
        adj = Chem.GetAdjacencyMatrix(mol)
        coo_adj = coo_matrix(adj)
        edge_index = torch.tensor([coo_adj.row, coo_adj.col], dtype=torch.long)
        self.feature_dict['edge_index'] = edge_index

        # 边特征（与 edge_index 保持顺序一致）
        bond_features = []
        if self.bond_features and edge_index.size(1) > 0:
            for i, j in zip(coo_adj.row, coo_adj.col):
                bond = mol.GetBondBetweenAtoms(int(i), int(j))
                if bond is not None:
                    bond_features.append(self._bond_featurizer(bond))
                else:
                    bond_features.append([0] * 6)  # 用零向量填补无效边
        else:
            bond_features = []

        self.feature_dict['edge_attr'] = bond_features

        return self.feature_dict

# 数据集类
class MolDataset:
    def __init__(self, csv_path, save_path='./processed_dataset.pt'):
        self.csv_path = csv_path
        self.save_path = save_path
        self.featurizer = MoleculeFeaturizer()
        self.data_list = []


        self._process_csv()

    def _process_csv(self):
        try:
            df = pd.read_csv(self.csv_path)
            print(f"Successfully loaded CSV file from {self.csv_path}")
        except Exception as e:
            print(f"Error loading CSV file: {e}")
            return

        if not all(col in df.columns for col in ['Smiles', 'logSw']):
            print("CSV file must contain 'Smiles' and 'logSw' columns")
            return

        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing molecules"):
            smiles = row['Smiles']
            label = row['logSw']

            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                print(f"Skipping invalid SMILES: {smiles}")
                continue

            try:
                features = self.featurizer(mol)

                data = Data()
                data.smiles = smiles
                data.y = torch.tensor([label], dtype=torch.float32)
                data.x = torch.tensor(features['x'], dtype=torch.float32)
                data.edge_index = features['edge_index']

                if len(features['edge_attr']) > 0:
                    data.edge_attr = torch.tensor(features['edge_attr'], dtype=torch.float32)
                else:
                    data.edge_attr = torch.empty((0, 6), dtype=torch.float32)

                self.data_list.append(data)
            except Exception as e:
                print(f"Error processing molecule {smiles}: {e}")
                continue

        if self.data_list:
            torch.save(self.data_list, self.save_path)
            print(f"Processed dataset saved to {self.save_path}")
        else:
            print("No valid molecules processed")

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

# 启动数据处理流程
if __name__ == "__main__":
    csv_path = '/content/SweetpredDB.csv'
    dataset = MolDataset(csv_path)

    if len(dataset) > 0:
        print(f"\n数据集包含 {len(dataset)} 个有效分子")
        sample = dataset[0]
        print("\n第一个分子信息:")
        print(f"SMILES: {sample.smiles}")
        print(f"甜度值: {sample.y.item()}")
        print(f"原子特征维度: {sample.x.shape}")
        print(f"边索引维度: {sample.edge_index.shape}")
        if hasattr(sample, 'edge_attr'):
            print(f"边特征维度: {sample.edge_attr.shape}")
    else:
        print("数据集为空，请检查输入文件")


### **分子图数据预处理原理解释**



本项目的核心目标是**将SMILES格式的分子数据转化为适用于图神经网络（GNN）模型的数据结构**，用于预测分子的甜度（logSw）。这个预处理流程是图神经网络模型的关键步骤，它将分子表示从字符串（SMILES）转换为图结构，从而能够利用图结构的表达能力和GNN的强大学习能力建模分子的结构-性质关系。

---

#### 一、为什么用图结构表示分子？

在化学中，一个分子可以被看作是一个无向图，其中：
- **原子是节点（nodes）**
- **共价键是边（edges）**

而**图神经网络（GNN）**正是专为这种非欧几里得结构（如图结构）设计的深度学习模型。通过在图上进行消息传递（Message Passing），GNN能有效学习每个节点（原子）在其邻居环境下的表示，并最终聚合为整个分子的全局表示，用于预测分子的性质。

---

#### 二、SMILES到分子图的转换过程

项目中使用了`RDKit`来将SMILES字符串转换为分子图。RDKit 是化学信息学中常用的工具包，它可以：
- 将SMILES解析为分子结构
- 获取原子的性质，如原子序数、杂化状态、手性等
- 获取原子之间的键类型，如单键、双键、芳香键等

---

#### 三、节点（原子）特征设计

节点特征是GNN的输入之一，直接决定模型能否充分理解原子化学属性。本项目中每个原子的特征是**多个离散属性的one-hot编码拼接**，包括：

| 特征 | 含义 | 化学意义 |
|------|------|----------|
| 原子序数 | 元素种类（如C, O, N等） | 决定原子的基本属性，如电负性、成键能力 |
| 原子度（degree） | 与原子相连的键数 | 表征原子的连接性，如端基、中间基团 |
| 杂化轨道类型 | sp、sp2、sp3等 | 反映电子排布，决定分子的立体结构 |
| 手性标记 | S/R或None | 表示手性中心，影响分子的立体异构性 |
| 氢原子数 | 与原子绑定的氢数 | 影响极性、分子大小等 |
| 芳香性 | 是否为芳香环结构的一部分 | 芳香环具有特殊稳定性，影响化学活性 |

**这些特征有助于神经网络捕捉原子在分子中所处的化学环境。**

---

#### 四、边（键）特征设计

图结构中的边表示原子之间的共价键。边特征同样采用one-hot形式，编码了以下信息：

| 特征 | 含义 |
|------|------|
| 键类型 | 单键、双键、三键、芳香键 |
| 是否共轭 | 与π电子离域有关，如苯环 |
| 是否在环内 | 判断该键是否构成环结构 |

这些特征有助于模型判断结构中的**π-共轭系统、环张力**等对分子性质的重要影响。

---

#### 五、构造 PyG 所需的图数据对象

最终，每个分子被表示为`torch_geometric.data.Data`对象，其中包含：
- `x`: 原子特征矩阵，形状为`[原子数, 特征维数]`
- `edge_index`: 边的索引，形状为`[2, 边数]`，用两个数组分别表示每条边的起点和终点原子
- `edge_attr`: 边的特征矩阵（可选）
- `y`: 标签值，即目标性质（如甜度）

这样的表示可以被**PyTorch Geometric（PyG）**直接用于训练图神经网络模型，如 GCN、GAT、GIN 等。

---

#### 六、与深度学习的连接

构建完图结构后，我们可以使用深度神经网络对这些图进行建模：

1. **图卷积层（GNN layers）**会根据边的结构和边特征，学习每个原子的隐空间表示。
2. **图池化（global pooling）**会把原子的表示整合成整个分子的表示。
3. **全连接层（MLP）**将最终图向量映射到目标属性（logSw）上，实现分子性质的预测。

这种方式相比传统的手工特征工程，具有更强的泛化能力和端到端学习能力。

---

#### 七、小结

本数据处理过程实现了：
- 从化学SMILES字符串自动生成图结构
- 保留并编码关键的化学属性信息
- 构造适合图神经网络模型训练的数据集

为后续的图神经网络模型打下坚实基础，实现从分子结构到性质的**深度建模与精准预测**。这就是图神经网络在化学中的魅力所在。


### **为什么要使用One-Hot编码，而不是直接使用原始离散属性？**



在深度学习中，数据的预处理和特征工程对模型的表现至关重要。尤其是在处理像化学分子这种复杂的结构数据时，如何有效地表示离散属性（例如原子的类型、杂化状态、键的类型等）是一个重要的问题。为了使模型能够有效地学习这些离散属性，我们通常选择使用**One-Hot编码**，而不是直接使用原始的离散属性值。以下是几个主要原因：

---

#### 一、深度学习模型的需求

深度学习模型，尤其是神经网络，倾向于处理数值数据，因为数值数据的大小关系可以通过数学运算（如加法、乘法等）捕捉和利用。对于离散属性：
- **原始离散属性**（如原子类型编号、杂化类型编号）通常是整数或者分类标识符，它们之间并没有显著的顺序关系。例如，原子序数（1代表氢，6代表碳）本质上只是一个标签，而不是“1比6大”这样有意义的数值关系。
- **One-Hot编码**将这些离散的类别转换为一个向量，其中每个类别都对应一个特定位置的1，而其他位置是0。这样，模型不会误解这些离散类别之间的“大小关系”，因为One-Hot编码的所有类别之间是**等距的**，即它们之间没有内在的数学顺序关系。

通过One-Hot编码，我们有效避免了模型学习到错误的关系，例如错误地认为氢（编号1）和碳（编号6）之间的数值差距有实际意义。

---

#### 二、避免模型误解类别间的顺序关系

假设我们直接将原始的离散属性（例如原子类型）作为模型输入，如果我们用整数表示原子类型（例如，氢是1，氮是2，碳是3），神经网络可能会**误解**这些整数之间存在某种顺序关系，比如：
- 数字“1”可能在模型内部被视为比“6”小，或者“6”与“1”之间有某种距离（例如6和1之间差值是5）。
  
这种关系对原子类型来说是没有意义的，因为原子类型之间没有自然的顺序。实际上，氢（1）和碳（6）是完全不同的元素，直接用数字表示会导致模型在学习时引入错误的假设。

**One-Hot编码**通过将每个类别表示为一个独立的二进制向量，消除了这种误解，使得每个类别都是独立的。比如：
- 氢会被表示为[1, 0, 0, ..., 0]（在原子类型特征向量中只有氢的对应位置是1，其他位置是0）
- 碳会被表示为[0, 0, 1, ..., 0]
  
这意味着模型只能通过学习每个类别对应的单独维度来判断原子的类型，而不会错误地学习到它们之间的大小关系。

---

#### 三、简化模型的学习过程

使用One-Hot编码的另一个优势是**简化了模型的学习过程**。One-Hot编码使得每个离散属性的每个取值都能独立地进行学习，而不必担心这些取值之间的潜在关系。
- 如果我们直接使用原始的离散属性（例如整数编码），模型必须自动学习这些属性之间的关系，可能会导致较长的训练时间或性能下降。
- One-Hot编码通过为每个类别创建独立的特征维度，**明确地将类别之间的关系隔离开**，让模型在学习时专注于如何区分每个类别的特征，而不需要担心类别之间的潜在数值关系。

例如，在化学分子中，原子类型、杂化状态、手性等各自是独立的特征，One-Hot编码能够让模型专注于从这些独立特征中学习更复杂的模式，而不会因为类别间的错误顺序而影响性能。

---

#### 四、提高模型的泛化能力

One-Hot编码有助于提高模型的**泛化能力**，尤其是在面对新的、未见过的类别时。如果我们使用原始的离散属性（例如整数），一旦模型遇到一个没有在训练集出现过的值，模型可能无法正确理解这个值并进行有效预测。而One-Hot编码通过将每个类别独立表示为一个向量，使得模型能够在面对新的类别时仍然能够进行有效的学习，因为每个类别都有一个明确的表示。

---

#### 五、总结

通过使用One-Hot编码，我们实现了：
- **消除无关顺序关系**，防止模型错误学习类别之间的数值关系；
- **简化模型的学习过程**，让模型更专注于每个特征；
- **提高模型的泛化能力**，让模型更好地处理新类别。

总的来说，One-Hot编码是一种在深度学习中广泛使用的技术，它能够有效地处理离散属性，并为模型提供清晰、无歧义的输入，从而增强模型的表达能力和预测能力。

### **让我们随机找一个数据瞅瞅它长什么样**

In [ ]:
import random
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import torch
import numpy as np
from torch_geometric.utils import to_networkx, to_dense_adj
from rdkit.Chem import Draw, AllChem
from google.colab import data_table

# 启用 Colab 的交互式表格显示，并设置不做截断
data_table.enable_dataframe_formatter()
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def visualize_and_inspect_aligned_molecule(dataset):
    if len(dataset) == 0:
        print("数据集为空，无法进行可视化。")
        return

    # 1. 筛选重原子数小于 15 的分子 (保持图表清晰)
    small_molecules_indices = []
    for i in range(len(dataset)):
        if dataset[i].x.shape[0] < 15:
            small_molecules_indices.append(i)

    if not small_molecules_indices:
        print("未找到符合条件的分子。")
        return

    # 2. 随机抽取一个符合条件的样本
    idx = random.choice(small_molecules_indices)
    data = dataset[idx]

    print(f"\n{'='*40}")
    print(f"随机抽取的样本索引: {idx}")
    print(f"SMILES: {data.smiles}")
    print(f"甜度值 (Label): {data.y.item()}")
    print(f"原子数量: {data.x.shape[0]}")
    print(f"{'='*40}\n")

    # =========================================
    # 可视化部分 (图像 - 坐标对齐版)
    # =========================================
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # --- 准备 RDKit 分子对象并计算 2D 坐标 ---
    mol = Chem.MolFromSmiles(data.smiles)
    if mol:
        # 强制计算 2D 坐标，这样我们才能获取每个原子的 (x, y) 位置
        AllChem.Compute2DCoords(mol)

        # 给原子打上索引标签
        for i, atom in enumerate(mol.GetAtoms()):
            atom.SetProp('molAtomMapNumber', str(i))

        # 绘制左图：RDKit 标准化学结构
        img = Draw.MolToImage(mol, size=(500, 500))
        axes[0].imshow(img)
        axes[0].set_title(f"RDKit Chemical Structure\n(Index: {idx})", fontsize=12, fontweight='bold')
        axes[0].axis('off')

        # --- 获取 RDKit 坐标用于 NetworkX ---
        # RDKit 的 Conformer 存储了原子的空间位置
        conf = mol.GetConformer()
        rdkit_pos = {}
        for i in range(mol.GetNumAtoms()):
            # 获取第 i 个原子的位置
            pt = conf.GetAtomPosition(i)
            # 存入字典 {节点索引: (x坐标, y坐标)}
            rdkit_pos[i] = (pt.x, pt.y)
    else:
        axes[0].text(0.5, 0.5, "Invalid Molecule", ha='center')
        rdkit_pos = None

    # --- 右图：NetworkX 图结构 (使用 RDKit 坐标) ---
    G = to_networkx(data, to_undirected=True)

    axes[1].set_title("NetworkX Graph Topology\n(Aligned with RDKit Coordinates)", fontsize=12, fontweight='bold')

    if rdkit_pos:
        # 使用自定义的 pos (来自 RDKit) 替代 spring_layout
        nx.draw(G, pos=rdkit_pos, ax=axes[1],
                with_labels=True,
                node_color='lightblue',
                node_size=600,
                edge_color='gray',
                width=2, # 增加边宽
                font_weight='bold')

        # 稍微调整一下坐标轴范围，防止节点贴边
        x_values, y_values = zip(*rdkit_pos.values())
        x_margin = (max(x_values) - min(x_values)) * 0.1
        y_margin = (max(y_values) - min(y_values)) * 0.1
        axes[1].set_xlim(min(x_values) - x_margin, max(x_values) + x_margin)
        axes[1].set_ylim(min(y_values) - y_margin, max(y_values) + y_margin)
    else:
        # 如果获取坐标失败，回退到随机布局
        nx.draw(G, ax=axes[1], with_labels=True)

    plt.tight_layout()
    plt.show()

    # =========================================
    # 矩阵数据展示 (Colab 表格)
    # =========================================
    print("\n" + "="*20 + " 1. 节点特征矩阵 (Node Features X) " + "="*20)
    node_cols = [f"Feature_{i}" for i in range(data.x.shape[1])]
    df_x = pd.DataFrame(data.x.numpy().astype(int), columns=node_cols)
    df_x.index.name = "Atom_Index"
    display(data_table.DataTable(df_x, num_rows_per_page=20))

    print("\n" + "="*20 + " 2. 邻接矩阵 (Adjacency Matrix A) " + "="*20)
    adj_dense = to_dense_adj(data.edge_index)[0].numpy().astype(int)
    cols_adj = [f"Atom_{i}" for i in range(adj_dense.shape[0])]
    df_adj = pd.DataFrame(adj_dense, columns=cols_adj, index=cols_adj)
    display(data_table.DataTable(df_adj, num_rows_per_page=20))

    print("\n" + "="*20 + " 3. 边特征矩阵 (Edge Features E) " + "="*20)
    if data.edge_attr is not None and data.edge_attr.size(0) > 0:
        edge_cols = ["Is_Single", "Is_Double", "Is_Triple", "Is_Aromatic", "Is_Conjugated", "Is_InRing"]
        src, dst = data.edge_index
        edge_data = data.edge_attr.numpy().astype(int)

        df_edge = pd.DataFrame(edge_data, columns=edge_cols)
        df_edge.insert(0, "Target_Atom", dst.numpy())
        df_edge.insert(0, "Source_Atom", src.numpy())

        display(data_table.DataTable(df_edge, num_rows_per_page=20))
    else:
        print("该分子没有边或边特征为空。")

# 运行
if __name__ == "__main__":
    if 'dataset' in locals():
        visualize_and_inspect_aligned_molecule(dataset)
    else:
        print("请先运行数据处理代码生成 dataset。")

### 数据解释

#### 1\. 节点特征矩阵 `data.x` (维度: 69)

该矩阵每一行代表一个原子。由于多个 One-Hot 向量是首尾拼接的，我们需要累加索引来定位特征。总维度为 **69** (38 + 6 + 9 + 9 + 6 + 1)。

| 全局索引范围 | 特征类别 | 长度 | 具体每一位 (Bit) 的详细对应 |
| :--- | :--- | :--- | :--- |
| **0 - 37** | **原子类型**<br>(Atom Type) | 38 | **0-36**: 对应以下 37 种元素：<br>H, Li, Be, B, C, N, O, F, Na, Mg, Al, Si, P, S, Cl, K, Ca, Sc, Ni, Cu, Zn, Ga, Ge, As, Se, Br, Kr, Rb, Sr, Pd, Ag, Cd, In, Sn, Sb, Te, I<br>**37**: 其他元素 (不在上述列表中) |
| **38 - 43** | **连接度**<br>(Degree) | 6 | **38**: 连接度 0 (孤立原子)<br>**39**: 连接度 1<br>**40**: 连接度 2<br>**41**: 连接度 3<br>**42**: 连接度 4<br>**43**: 其他 (连接度 \> 4) |
| **44 - 52** | **杂化类型**<br>(Hybridization) | 9 | **44**: UNSPECIFIED (未指定)<br>**45**: S<br>**46**: SP<br>**47**: SP2<br>**48**: SP3<br>**49**: SP2D<br>**50**: SP3D<br>**51**: SP3D2<br>**52**: 其他 (含 OTHER) |
| **53 - 61** | **手性标签**<br>(Chiral Tag) | 9 | **53**: CHI\_UNSPECIFIED (无/未指定)<br>**54**: CHI\_CW (顺时针)<br>**55**: CHI\_CCW (逆时针)<br>**56**: CHI\_OTHER<br>**57**: CHI\_TETRAHEDRAL<br>**58**: CHI\_ALLENE<br>**59**: CHI\_SQUAREPLANAR<br>**60**: CHI\_TRIGONALBIPYRAMIDAL<br>**61**: 其他 (含 OCTAHEDRAL) |
| **62 - 67** | **氢原子数**<br>(Num Hs) | 6 | **62**: 0 个氢<br>**63**: 1 个氢<br>**64**: 2 个氢<br>**65**: 3 个氢<br>**66**: 4 个氢<br>**67**: 其他 (氢原子 \> 4) |
| **68** | **芳香性**<br>(Is Aromatic) | 1 | **68**: 是否为芳香原子 (1=是, 0=否) |

-----

#### 2\. 边特征矩阵 `data.edge_attr` (维度: 6)

边特征矩阵每一行代表一条化学键。索引从 0 开始。

| 索引位置 | 特征名称 | 含义 (如果该位为 1) |
| :--- | :--- | :--- |
| **0** | **单键** (Single) | 该键是单键 |
| **1** | **双键** (Double) | 该键是双键 |
| **2** | **三键** (Triple) | 该键是三键 |
| **3** | **芳香键** (Aromatic) | 该键是芳香键 (常见于苯环) |
| **4** | **共轭** (Is Conjugated) | 该键处于共轭体系中 (单双键交替) |
| **5** | **在环中** (Is In Ring) | 该键是环结构的一部分 |

-----



## **2-GNN模型搭建、训练及评估**

### **先只用试试GAT（图注意力）网络**

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.loader import DataLoader
from sklearn.metrics import r2_score
import random
import numpy as np

# 设置随机种子以保证可复现性
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything()

# 图注意力网络
class GATNet(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=16, output_dim=1, heads=2):
        super(GATNet, self).__init__()
        self.gat1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=0.1)
        self.gat2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, dropout=0.1)
        self.pool = global_mean_pool  # 可改为 max_pool 等

        self.linear = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index, batch, edge_attr=None):
        x = self.gat1(x, edge_index)
        x = F.elu(x)
        x = self.gat2(x, edge_index)
        x = F.elu(x)
        x = self.pool(x, batch)
        out = self.linear(x)
        return out.squeeze()  # 输出一维预测值

# 加载数据

dataset = MolDataset(csv_path='/content/SweetpredDB.csv')

# 数据划分
train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# 模型准备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GATNet(input_dim=dataset[0].x.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
loss_fn = torch.nn.MSELoss()

# 训练函数
def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        pred = model(batch.x, batch.edge_index, batch.batch)
        loss = loss_fn(pred, batch.y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch.num_graphs
    return total_loss / len(train_loader.dataset)

# 验证函数
def evaluate(loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            pred = model(batch.x, batch.edge_index, batch.batch)
            preds.append(pred.cpu())
            labels.append(batch.y.cpu())
    preds = torch.cat(preds)
    labels = torch.cat(labels)
    mse = F.mse_loss(preds, labels).item()
    r2 = r2_score(labels.numpy(), preds.numpy())
    return mse, r2


import matplotlib.pyplot as plt

# 添加这两个列表用于记录loss
# ⏳ 训练主循环 + 保存特征
train_losses = []
val_losses = []
best_val_r2 = -float("inf")
best_epoch = 0  # 记录最佳模型的epoch

for epoch in range(1, 201):
    train_loss = train()
    val_mse, val_r2 = evaluate(val_loader)

    train_losses.append(train_loss)
    val_losses.append(val_mse)

    if val_r2 > best_val_r2:
        best_val_r2 = val_r2
        best_epoch = epoch  # 更新最佳模型的epoch
        torch.save(model.state_dict(), "Best_GAT_Model.pt")

    # 保存本轮训练的图表示 + 标签

    if epoch % 1 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Train Loss: {train_loss:.4f} | Val MSE: {val_mse:.4f} | Val R2: {val_r2:.4f}")

print(f"训练完成 ✅ 最佳验证集 R²: {best_val_r2:.4f} (在第 {best_epoch} 轮)")

In [ ]:
# 📈 绘制学习曲线
plt.figure(figsize=(5, 5),dpi=200)

# 指定 RGB 颜色（0~1 归一化）
train_color = (75 / 255, 116 / 255, 178 / 255)   # 蓝色
val_color = (219 / 255, 49 / 255, 36 / 255)      # 红色

plt.plot(train_losses, label='Train Loss', color=train_color)
plt.plot(val_losses, label='Validation Loss', color=val_color)

# 标记最佳模型的迭代次数
plt.axvline(x=best_epoch-1, color='black', linestyle='--', label=f'Best Model at Epoch {best_epoch}')

plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('GAT')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("loss_curve.png")
plt.show()



In [ ]:

# 绘制预测值和真实值比较图
best_model = torch.load("/content/Best_GAT_Model.pt")
model.load_state_dict(best_model)

model.eval()
preds, labels = [], []
with torch.no_grad():
    for batch in val_loader:
        batch = batch.to(device)
        pred = model(batch.x, batch.edge_index, batch.batch)
        preds.append(pred.cpu())
        labels.append(batch.y.cpu())

preds = torch.cat(preds)
labels = torch.cat(labels)


# 绘制预测值和真实值比较图
plt.figure(figsize=(5, 5), dpi=200)

# 绘制散点图，预测值用蓝色
plt.scatter(labels, preds, alpha=0.7, color=(75 / 255, 116 / 255, 178 / 255), label='Predictions')

# 绘制对角线，真实值用红色
plt.plot([min(labels), max(labels)], [min(labels), max(labels)], 'k--', lw=2, color=(219 / 255, 49 / 255, 36 / 255), label='True Values')

# 计算 R2 和 MSE
mse = F.mse_loss(preds, labels).item()
r2 = r2_score(labels.numpy(), preds.numpy())

# 添加标签、标题以及计算的 R2 和 MSE 值
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title(f'True vs Predicted Values (Best Model)\nMSE: {mse:.4f} | R²: {r2:.4f}')
plt.grid(True)
plt.tight_layout()
plt.legend()
plt.show()

### **GAT图神经网络训练与评估过程解释**



本项目旨在利用**图注意力网络（GAT）**来预测分子的甜度（logSw）。以下是对整个训练与评估过程的详细解释，包括如何构建GAT模型、训练、验证以及性能评估。

---

#### 一、图注意力网络（GAT）简介

图注意力网络（GAT）是一种基于图的深度学习模型，主要用于处理图结构数据。与传统的图卷积网络（GCN）不同，GAT采用了**注意力机制**来计算节点之间的信息传递权重，允许模型根据不同邻居节点的重要性自动调整权重，从而更灵活地进行图结构数据的学习。

在本项目中，GAT被用来处理分子图数据，模型的目标是从分子图中学习结构信息，以预测每个分子的甜度值（logSw）。

---

#### 二、模型架构

我们使用了一个包含两层GAT卷积层的网络，详细结构如下：

- **第一层GATConv**：接收原子特征输入，将其映射到一个中间的隐藏维度，使用多头注意力机制（`heads=2`），即每个节点会有两个注意力头进行信息汇聚。该层的输出会经过ReLU激活函数（`F.elu`）进行非线性变换。
  
- **第二层GATConv**：进一步对第一层的输出进行处理，使用一个注意力头（`heads=1`）。同样，输出会经过激活函数进行处理。

- **池化层**：使用`global_mean_pool`进行全图池化，它将图中所有节点的特征通过平均值汇聚成一个全局图表示。

- **全连接层**：最后通过一个线性层将图表示映射到预测目标（即甜度值），输出是一个单一的数值。

模型架构如下所示：
```python
class GATNet(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=16, output_dim=1, heads=2):
        # 定义GAT层、池化层和全连接层
        ...
    def forward(self, x, edge_index, batch, edge_attr=None):
        # 前向传播
        ...
```

---

#### 三、数据准备

我们使用了一个包含分子SMILES字符串和甜度值（logSw）的数据集，通过以下步骤准备数据：
1. **SMILES转分子图**：将SMILES字符串转换为分子图（每个分子图包含原子节点和键的连接信息）。
2. **特征提取**：为每个分子计算原子特征和键特征，并将其转化为图神经网络能够处理的格式。
3. **数据划分**：将数据集划分为训练集和验证集（80%用于训练，20%用于验证）。

通过`DataLoader`加载数据，以批次形式进行训练和验证。

---

#### 四、训练过程

在训练过程中，网络使用**均方误差损失函数（MSELoss）**来优化模型，目标是最小化预测值和真实值之间的差距。优化器选择了**Adam**，其具有自适应学习率的特性，有助于加速收敛。

训练过程中，每个batch的前向传播步骤包括：
1. 输入分子图的原子特征、边索引和批次信息。
2. 通过两层GAT卷积层提取特征。
3. 使用池化层对图进行汇聚，得到全图的表示。
4. 通过全连接层输出预测值。

在训练时，我们记录了每轮的训练损失（MSE）和验证集的**均方误差（MSE）**以及**决定系数（R²）**，以评估模型的性能。

训练代码片段如下：
```python
def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        pred = model(batch.x, batch.edge_index, batch.batch)
        loss = loss_fn(pred, batch.y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch.num_graphs
    return total_loss / len(train_loader.dataset)
```

---

#### 五、验证过程

在每轮训练之后，我们使用验证集来评估模型性能。模型在验证集上的表现通过**MSE损失和R²分数**来衡量：
- **MSE（均方误差）**：表示预测值和真实值之间的差距，数值越小表示模型越精确。
- **R²分数**：衡量模型对目标变量的解释能力，R²值越接近1，说明模型的拟合效果越好。

验证过程代码如下：
```python
def evaluate(loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            pred = model(batch.x, batch.edge_index, batch.batch)
            preds.append(pred.cpu())
            labels.append(batch.y.cpu())
    preds = torch.cat(preds)
    labels = torch.cat(labels)
    mse = F.mse_loss(preds, labels).item()
    r2 = r2_score(labels.numpy(), preds.numpy())
    return mse, r2
```

---

#### 六、训练与验证的迭代过程

在每个epoch的训练过程中，我们记录了训练损失和验证集的性能，具体的训练过程如下：

1. 每轮训练后，我们输出当前的训练损失、验证集的MSE和R²。
2. 如果验证集上的R²分数提高，我们会保存当前最好的模型。

训练主循环如下：
```python
for epoch in range(1, 201):
    loss = train()
    val_mse, val_r2 = evaluate(val_loader)

    train_losses.append(loss)
    val_losses.append(val_mse)

    if val_r2 > best_val_r2:
        best_val_r2 = val_r2
        torch.save(model.state_dict(), "best_gat_model.pt")

    if epoch % 1 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Val MSE: {val_mse:.4f} | Val R2: {val_r2:.4f}")
```

---

#### 七、最终结果

在训练完成后，我们打印出**最佳验证集R²**，这表示我们模型在验证集上达到了最佳的拟合效果。模型的训练过程中通过不断调整参数和优化过程，最终得到了一个能够较好预测分子甜度值的模型。

---

#### 八、总结

通过上述步骤，我们成功地实现了一个基于GAT的图神经网络模型，能够从分子的结构信息中学习到有用的特征，并预测分子的甜度值。训练过程的关键在于通过优化GAT模型的参数，使得网络能够学习到分子结构和目标性质之间的关系，并通过验证集的评估来确保模型的泛化能力。

### **接着试试GAT（图注意力）网络+MLP+边特征**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.loader import DataLoader
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import numpy as np
import random
import os

# ==========================================
# 1. 基础设置
# ==========================================
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything()

# 权重初始化函数
def init_weight(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

# ==========================================
# 2. 修改后的 MLP 模块 (支持传入 dropout)
# ==========================================
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1, dropout=0.4): # 默认改为 0.4
        super(MLP, self).__init__()
        # 使用 nn.Sequential，结构清晰
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),           # ❗ 修改：使用传入的高 dropout
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),           # ❗ 修改：使用传入的高 dropout
            nn.Linear(hidden_dim // 2, output_dim) # 回归输出层
        )

        # 应用权重初始化
        self.net.apply(init_weight)

    def forward(self, x):
        return self.net(x)

# ==========================================
# 3. GAT 模型 (包含边特征处理 + 强正则化)
# ==========================================
class GATNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=16, mlp_hidden=16, output_dim=1, heads=2, dropout=0.4, edge_dim=None):
        super(GATNet, self).__init__()

        # GAT 层 1
        # ❗ 修改：dropout 设为 0.4
        self.gat1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=dropout, edge_dim=edge_dim)

        # GAT 层 2
        self.gat2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, dropout=dropout, edge_dim=edge_dim)

        # 池化层
        self.pool = global_mean_pool

        # MLP 预测头
        # ❗ 修改：将 dropout 传递给 MLP
        self.mlp = MLP(input_dim=hidden_dim, hidden_dim=mlp_hidden, output_dim=output_dim, dropout=dropout)

    def forward(self, x, edge_index, batch, edge_attr=None):
        # 1. GAT 提取特征
        x = self.gat1(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)

        x = self.gat2(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)

        # 2. 全局池化 (获得图级别的表示)
        x = self.pool(x, batch)

        # 3. MLP 回归预测
        out = self.mlp(x)
        return out.squeeze() # 移除多余维度

    def extract_graph_embeddings(self, x, edge_index, batch, edge_attr=None):
        """用于 t-SNE 可视化的特征提取函数"""
        x = self.gat1(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)
        x = self.gat2(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)
        pooled = self.pool(x, batch)
        return pooled

# ==========================================
# 4. 数据加载与训练流程
# ==========================================

# 检查 dataset 是否存在
if 'dataset' not in locals():
    dataset = MolDataset(csv_path='/content/SweetpredDB.csv')

# 划分数据集
train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# 设备设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 获取维度信息
input_dim = dataset[0].x.shape[1]
if len(dataset) > 0 and dataset[0].edge_attr is not None:
    edge_dim = dataset[0].edge_attr.shape[1]
    print(f"✅ 检测到边特征维度: {edge_dim}")
else:
    edge_dim = None
    print("❌ 未检测到边特征")

# 初始化模型
# ❗ 策略调整：
# 1. Dropout 提高到 0.4 (Strong Regularization)
# 2. hidden_dim 保持较小 (16)，避免参数过多
model = GATNet(
    input_dim=input_dim,
    hidden_dim=16,
    mlp_hidden=128,
    edge_dim=edge_dim,
    dropout=0.4          # <--- 关键修改：高 Dropout
).to(device)

# ❗ 策略调整：增大 Weight Decay
# 从 1e-4 增加到 1e-3，惩罚过大的权重，防止过拟合
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=2e-3)
loss_fn = nn.MSELoss()

# 训练函数
def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        pred = model(batch.x, batch.edge_index, batch.batch, edge_attr=batch.edge_attr)
        loss = loss_fn(pred, batch.y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * batch.num_graphs
    return total_loss / len(train_loader.dataset)

# 验证函数
def evaluate(loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            pred = model(batch.x, batch.edge_index, batch.batch, edge_attr=batch.edge_attr)
            preds.append(pred.cpu())
            labels.append(batch.y.cpu())
    preds = torch.cat(preds)
    labels = torch.cat(labels)
    mse = F.mse_loss(preds, labels).item()
    r2 = r2_score(labels.numpy(), preds.numpy())
    return mse, r2

# 提取特征函数
def extract_global_features(loader):
    model.eval()
    all_features = []
    all_labels = []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            pooled = model.extract_graph_embeddings(batch.x, batch.edge_index, batch.batch, edge_attr=batch.edge_attr)
            all_features.append(pooled.cpu())
            all_labels.append(batch.y.cpu())
    all_features = torch.cat(all_features).numpy()
    all_labels = torch.cat(all_labels).numpy()
    return all_features, all_labels

# 保存目录
save_dir = "mol_global_features"
os.makedirs(save_dir, exist_ok=True)

# 训练循环
train_losses = []
val_losses = []
best_val_r2 = -float("inf")
best_epoch = 0

print("🚀 开始训练 GAT (Edge Features + Strong Regularization) ...")
print("配置: Dropout=0.4, Weight_Decay=1e-3")

for epoch in range(1, 201):
    train_loss = train()
    val_mse, val_r2 = evaluate(val_loader)

    train_losses.append(train_loss)
    val_losses.append(val_mse)

    if val_r2 > best_val_r2:
        best_val_r2 = val_r2
        best_epoch = epoch
        torch.save(model.state_dict(), "Best_GATEdge+MLP_Model.pt")

    # 保存特征
    global_feats, labels = extract_global_features(train_loader)
    np.save(os.path.join(save_dir, f"features_epoch_{epoch:03d}.npy"), global_feats)
    np.save(os.path.join(save_dir, f"labels_epoch_{epoch:03d}.npy"), labels)

    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | Train Loss: {train_loss:.4f} | Val MSE: {val_mse:.4f} | Val R2: {val_r2:.4f}")

print(f"\n✅ 训练完成。最佳验证集 R²: {best_val_r2:.4f} (在第 {best_epoch} 轮)")

In [ ]:
# 📈 绘制学习曲线
plt.figure(figsize=(5, 5),dpi=200)

# 指定 RGB 颜色（0~1 归一化）
train_color = (75 / 255, 116 / 255, 178 / 255)   # 蓝色
val_color = (219 / 255, 49 / 255, 36 / 255)      # 红色

plt.plot(train_losses, label='Train Loss', color=train_color)
plt.plot(val_losses, label='Validation Loss', color=val_color)

# 标记最佳模型的迭代次数
plt.axvline(x=best_epoch-1, color='black', linestyle='--', label=f'Best Model at Epoch {best_epoch}')

plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('GATEdge + MLP')
plt.legend()
plt.grid(True)
plt.tight_layout()
# plt.savefig("loss_curve.png")
plt.show()



In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# 1. 加载最佳模型权重
# 确保文件路径正确
best_model_path = "Best_GATEdge+MLP_Model.pt"
if os.path.exists(best_model_path):
    model.load_state_dict(torch.load(best_model_path))
    print("✅ 成功加载最佳模型权重")
else:
    print("❌ 未找到模型文件，请检查路径")

model.eval()
preds, labels = [], []

# 2. 推理 (注意加上 edge_attr)
with torch.no_grad():
    for batch in val_loader:
        batch = batch.to(device)

        # ❗❗❗ 关键修正：必须传入 edge_attr ❗❗❗
        pred = model(batch.x, batch.edge_index, batch.batch, edge_attr=batch.edge_attr)

        preds.append(pred.cpu())
        labels.append(batch.y.cpu())

preds = torch.cat(preds)
labels = torch.cat(labels)

# 3. 绘图
plt.figure(figsize=(4, 4), dpi=300)

# 绘制散点图 (蓝色)
plt.scatter(labels, preds, alpha=0.6, s=20,
            color=(75 / 255, 116 / 255, 178 / 255), label='Predictions')

# 绘制对角线 (红色)
min_val = min(labels.min(), preds.min())
max_val = max(labels.max(), preds.max())
plt.plot([min_val, max_val], [min_val, max_val], '--', lw=2,
         color=(219 / 255, 49 / 255, 36 / 255), label='Perfect Prediction')

# 计算 R2 和 MSE
mse = F.mse_loss(preds, labels).item()
r2 = r2_score(labels.numpy(), preds.numpy())

# 设置标题和标签
plt.xlabel('True Sweetness (logSw)')
plt.ylabel('Predicted Sweetness')
plt.title(f'True vs Predicted Values (Best Model)\nWith Edge Features\nMSE: {mse:.4f} | R²: {r2:.4f}')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()

plt.show()

### **GAT模型中加入MLP模块与边特征的作用**



在图神经网络（GNN）的设计中，如何有效地捕捉图中节点之间的复杂关系，以及如何在模型中融入更多的结构信息（如边的特征），是非常关键的。为了提高模型的表达能力，我们对**GAT网络**进行了扩展，加入了**MLP模块**和**边特征（edge attr）**。在本项目中，这些改进有助于提升模型的表现。下面将重点解释**为什么加MLP模块，边特征如何帮助提升模型，以及GAT中信息传递的过程和可学习的参数**。

---

#### 一、为何加MLP模块？

**MLP（多层感知机）**是深度学习中一种常用的全连接层网络，它通过将图神经网络的输出进行进一步的非线性变换，从而加强模型的拟合能力。在本模型中，GAT输出的是经过池化（`global_mean_pool`）后得到的分子级别的图表示（一个全局的向量）。这些表示本身仍然是**低维的、简单的**，还需要通过MLP进一步**学习和转化**，以便输出更精确的预测结果（如甜度）。

加入MLP的好处是：
- **非线性映射**：GAT虽然通过图卷积操作能够学习到图的结构信息，但如果没有MLP层，它的输出可能只是简单的线性变换。MLP通过更多的非线性层帮助模型学习更复杂的特征。
- **提升表达能力**：MLP通过多层网络结构能够捕捉更多的复杂特征，尤其是在多任务学习、回归任务等场景下，MLP能够在局部图表示的基础上进行进一步的特征提炼。

**为什么MLP在GAT之后加**：
- GAT能够提取每个节点的信息并在图中进行传播，但得到的表示是节点级的，对于分子来说，模型需要对所有节点进行汇聚并输出一个整体的分子级别的表示，MLP能够在此基础上进行更加精细的特征学习，最终得到一个更好的预测结果。

---

#### 二、为何使用边特征（edge_attr）？

在原始的GAT模型中，模型通过节点间的邻接矩阵来进行信息传递。但是，**边特征（edge_attr）**提供了更多的有用信息，如**键的类型**（单键、双键、芳香键等）、**是否共轭**等，这些对于描述分子的结构和化学性质至关重要。例如，芳香键在分子的稳定性、反应性等方面有着独特的影响，仅依靠原子节点的信息可能无法完全捕捉到这一点。

使用边特征的好处：
- **增强信息传递的灵活性**：边特征可以告诉GAT网络**不同类型的键**对节点之间的关系有何种影响。通过这种方式，模型能够更好地理解不同边类型对分子性质的影响，从而优化信息传递的权重。
- **提高模型表现**：边特征不仅能让模型更好地学习图的结构，还能帮助模型捕捉到分子中特定的化学性质，进而提高预测精度。

在本模型中，我们通过`edge_attr`将这些信息传递给GAT卷积层，并在每次图卷积操作时，让模型考虑边的特征。这种改进大大提高了模型对于复杂化学结构的理解能力。

---

#### 三、GAT模型中的信息传递与可学习参数

GAT模型通过**注意力机制**来进行信息传递，每个节点在与邻居节点交换信息时，并不是均匀地进行，而是根据每个邻居节点的重要性来动态地分配权重。具体过程如下：

1. **节点表示计算**：在GAT中，每个节点的表示是通过其邻居节点的表示加权求和得到的。具体来说，每个节点会对它的邻居节点使用**注意力权重**进行加权，并将这些加权后的信息聚合成该节点的新表示。这些权重是由**注意力机制**学习得出的，而不是固定的。

2. **注意力机制**：GAT的核心优势在于它的注意力机制，它能够通过**自适应的方式**对邻居节点进行加权。每个节点的注意力系数（即权重）是**可学习的**，这些系数会随着训练逐步调整，以便让模型能够聚焦于对最终任务（如甜度预测）有更大影响的邻居节点。

3. **每层的可学习参数**：在每一层GAT中，**权重矩阵**（即GAT的参数）和**注意力系数**（用于加权邻居节点的信息）是可学习的。这意味着，GAT中的信息传递过程并不是固定的，而是**动态调整的**，网络可以通过反向传播来优化这些参数，最终让模型能够更好地学习到节点和图结构之间的关系。

4. **每一层的GAT是否有MLP**：GAT层本身并不包括MLP，但它通过注意力机制实现了信息传递。它的本质是基于节点之间的关系来进行信息聚合，而不是简单的线性组合。因此，在GAT层之后加入MLP是一种增强表达能力的方式，使得网络能够在图卷积后进一步捕捉非线性关系。

---

#### 四、总结

- **加入MLP模块**是为了通过多层感知机增强模型的非线性表达能力，进一步精细化分子表示的学习。
- **边特征（edge_attr）**的引入使得模型能够在图卷积时考虑不同类型的化学键，从而更好地捕捉分子的化学结构信息，提高了模型的表现。
- **GAT中的信息传递**通过可学习的注意力机制动态调整邻居节点之间的加权，确保信息传递过程更加灵活，并通过反向传播优化注意力权重和权重矩阵，提升了模型的表达能力和预测精度。

最终，GAT与MLP结合，以及使用边特征，有效地提升了模型的预测性能，使其能够更好地处理分子图结构，并预测分子的甜度。

### **降维可视化不同迭代分子全局特征的变化**

#### t-sne降维可视化

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

epoch = 170  # 想看哪一轮就改哪个编号
features = np.load(f"mol_global_features/features_epoch_{epoch:03d}.npy")
labels = np.load(f"mol_global_features/labels_epoch_{epoch:03d}.npy")

# 执行 t-SNE 降维
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
embeds_2d = tsne.fit_transform(features)

# 自定义 RGB 颜色（归一化为0~1）
high_color = (219 / 255, 49 / 255, 36 / 255)   # 高甜 → 红色
low_color = (75 / 255, 116 / 255, 178 / 255)   # 低甜 → 蓝色

# 绘图
plt.figure(figsize=(5, 5),dpi=200)
plt.scatter(embeds_2d[labels > 3, 0], embeds_2d[labels > 3, 1], c=[high_color], label='High Sweetness (>3)', alpha=0.7)
plt.scatter(embeds_2d[labels <= 3, 0], embeds_2d[labels <= 3, 1], c=[low_color], label='Low Sweetness (≤3)', alpha=0.7)

plt.legend()
plt.title(f"t-SNE of Molecule Representations (Epoch {epoch})")
plt.grid(True)
plt.tight_layout()
plt.show()


#### t-sne降维可视化→gif

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from PIL import Image
from tqdm import tqdm

# 设置路径和参数
feature_dir = "mol_global_features"
output_dir = "tsne_images"
gif_path = "tsne_evolution.gif"
os.makedirs(output_dir, exist_ok=True)

# 自定义 RGB 颜色（归一化）
high_color = (219 / 255, 49 / 255, 36 / 255)   # 红色
low_color = (75 / 255, 116 / 255, 178 / 255)   # 蓝色

# 获取所有 epoch 文件，并每隔 5 轮采样一次
all_epochs = sorted([int(f.split("_")[-1].split(".")[0]) for f in os.listdir(feature_dir) if f.startswith("features")])
epochs = [e for e in all_epochs if e % 5 == 0 or e == all_epochs[-1]]  # 每5轮一个图，保留最后一轮

print(f"将处理 {len(epochs)} 个 epoch 的 t-SNE 可视化图像...")

# t-SNE 图生成（添加 tqdm 进度条）
for epoch in tqdm(epochs, desc="Running t-SNE and saving images"):
    features = np.load(f"{feature_dir}/features_epoch_{epoch:03d}.npy")
    labels = np.load(f"{feature_dir}/labels_epoch_{epoch:03d}.npy")

    tsne = TSNE(n_components=2, perplexity=30, random_state=42)
    embeds_2d = tsne.fit_transform(features)

    # 可视化
    plt.figure(figsize=(5, 5),dpi=200)
    plt.scatter(
        embeds_2d[labels > 3, 0], embeds_2d[labels > 3, 1],
        c=[high_color], label='High Sweetness (>3)', alpha=0.7
    )
    plt.scatter(
        embeds_2d[labels <= 3, 0], embeds_2d[labels <= 3, 1],
        c=[low_color], label='Low Sweetness (≤3)', alpha=0.7
    )
    plt.legend()
    plt.title(f"t-SNE of Molecule Representations\nEpoch {epoch}")
    plt.axis('off')
    plt.tight_layout()
    save_path = os.path.join(output_dir, f"tsne_epoch_{epoch:03d}.png")
    plt.savefig(save_path)
    plt.close()

print("✅ 所有 t-SNE 图像已生成")

# 生成GIF
print("📽️ 正在生成动图...")
image_files = [os.path.join(output_dir, f"tsne_epoch_{epoch:03d}.png") for epoch in epochs]
images = []
for f in tqdm(image_files, desc="Loading images for GIF"):
    images.append(Image.open(f))

images[0].save(gif_path, save_all=True, append_images=images[1:], duration=500, loop=0)

print(f"✅ 动图已保存为：{gif_path}")


### **t-SNE可视化与分子表示学习的原理**



在深度学习中，尤其是图神经网络（GNN）等模型用于分子表示学习时，模型的目标是将高维的分子特征映射到一个低维空间中，以便能够更好地进行预测或分类任务。t-SNE（t-分布随机邻域嵌入）是一种常用的降维技术，用于将高维数据映射到2维或3维空间，并且能够保留数据点之间的相对距离和局部结构。在本项目中，我们使用t-SNE对每个epoch训练过程中提取的分子特征进行可视化，从而观察模型训练过程中分子表示的变化。

---

#### 一、t-SNE可视化

t-SNE通过以下方式帮助我们理解模型训练的过程：
1. **降维**：t-SNE将高维的分子表示降维为二维（2D）空间，使我们可以在平面上直观地观察分子间的关系。降维后，数据点的位置保留了它们在高维空间中的相对关系。
2. **聚类**：t-SNE可以通过可视化显示哪些数据点（即分子表示）在低维空间中更靠近，从而帮助我们理解哪些分子在模型中被认为是相似的。

在每个epoch生成的t-SNE图中，我们将“高甜度”（`label > 3`）和“低甜度”（`label ≤ 3`）的分子使用不同的颜色区分开来。这让我们能够观察到随着训练过程的推进，高甜度和低甜度的分子是否在特征空间中逐渐分开，从而帮助我们判断模型是否成功地学习到了分子结构与甜度之间的关系。

---

#### 二、训练过程中高甜度和低甜度分子的分离

随着epoch数的增加，我们观察到在t-SNE图中，高甜度和低甜度的分子开始在特征空间中**逐渐分离**。这一现象反映了模型在训练过程中逐渐学习到更有效的分子表示，从而能够在特征空间中将不同类型的分子（高甜度和低甜度）区分开来。具体来说：
1. **初期训练阶段**：在训练的初期，模型可能还没有完全掌握如何区分不同类型的分子，因此高甜度和低甜度的分子可能分布较为混乱，或者它们的表示在特征空间中没有明显的区分。
2. **训练中期及后期**：随着训练的深入，模型通过优化参数逐渐学习到不同类型分子之间的特征差异，尤其是通过图卷积层和MLP层的非线性变换，模型能够逐渐将具有不同性质（如甜度）的分子映射到特征空间中的不同区域。
3. **最终阶段**：在训练的后期，高甜度和低甜度的分子开始明显分离，这表明模型已经学会了如何在特征空间中区分这些分子，说明模型已经很好地学习到了甜度预测任务中的模式和规律。

这种分离现象是**表示学习（Representation Learning）**的一个重要指标，表明模型在低维空间中提取了有效的、区分不同标签（高甜度和低甜度）的特征。

---

#### 三、t-SNE和表示学习的原理

t-SNE的工作原理基于以下几个关键步骤：
1. **局部结构保持**：t-SNE通过计算每对数据点之间的相似度，并试图在低维空间中保持这些相似度。这意味着，相似的分子在高维空间中相互接近，t-SNE会尽量将它们映射到低维空间中的相对接近位置。
2. **全局结构变换**：尽管t-SNE强调保持局部结构，但它也会对全局结构进行一定的变换，使得数据点的分布更符合低维空间的性质。最终，t-SNE的目标是尽可能让每个数据点在低维空间中的表示保留其在高维空间中的相对位置关系。

在训练过程中，图神经网络（GNN）会通过图卷积层对图中的节点进行信息传递，逐步从局部的原子信息推导出分子的全局表示。当我们将这些高维的分子表示输入到t-SNE中时，t-SNE会根据数据点间的相似度将这些表示映射到低维空间中。如果模型学习得当，标签相同的分子会被映射到低维空间的相邻区域，而不同标签的分子则会被映射到远离的位置。

---

#### 四、总结

通过对训练过程中每个epoch生成的t-SNE图进行可视化，我们可以直观地观察到模型在训练过程中的进步，特别是**高甜度和低甜度的分子逐渐分离**，这一现象表明模型已经学会了如何根据分子的结构特征来预测甜度值。这种分离现象反映了图神经网络在进行表示学习时，通过图卷积层的训练和MLP模块的进一步非线性变换，成功地将具有不同标签的分子映射到特征空间中的不同区域。

t-SNE的降维过程帮助我们验证了模型的学习效果，证明了随着训练的进行，分子之间的特征差异逐渐变得更加明显，从而提高了预测准确性。

#### PCA降维可视化→gif

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from PIL import Image
from tqdm import tqdm

# 设置路径和参数
feature_dir = "mol_global_features"          # 特征目录保持原样
output_dir = "pca_images"              # 输出图像文件夹
gif_path = "pca_evolution.gif"         # 输出动图名称
os.makedirs(output_dir, exist_ok=True)

# 自定义 RGB 颜色（归一化）
high_color = (219 / 255, 49 / 255, 36 / 255)   # 红色
low_color = (75 / 255, 116 / 255, 178 / 255)   # 蓝色

# 获取所有 epoch 文件，并每隔 5 轮采样一次
all_epochs = sorted([
    int(f.split("_")[-1].split(".")[0])
    for f in os.listdir(feature_dir)
    if f.startswith("features")
])
epochs = [e for e in all_epochs if e % 5 == 0 or e == all_epochs[-1]]  # 每5轮一个图，保留最后一轮

print(f"将处理 {len(epochs)} 个 epoch 的 PCA 可视化图像...")

# PCA 图生成（添加 tqdm 进度条）
for epoch in tqdm(epochs, desc="Running PCA and saving images"):
    features = np.load(f"{feature_dir}/features_epoch_{epoch:03d}.npy")
    labels = np.load(f"{feature_dir}/labels_epoch_{epoch:03d}.npy")

    pca = PCA(n_components=2)
    embeds_2d = pca.fit_transform(features)

    # 可视化
    plt.figure(figsize=(5, 5), dpi=200)
    plt.scatter(
        embeds_2d[labels > 3, 0], embeds_2d[labels > 3, 1],
        c=[high_color], label='High Sweetness (>3)', alpha=0.7
    )
    plt.scatter(
        embeds_2d[labels <= 3, 0], embeds_2d[labels <= 3, 1],
        c=[low_color], label='Low Sweetness (≤3)', alpha=0.7
    )
    plt.legend()
    plt.title(f"PCA of Molecule Representations\nEpoch {epoch}")
    plt.axis('off')
    plt.tight_layout()
    save_path = os.path.join(output_dir, f"pca_epoch_{epoch:03d}.png")
    plt.savefig(save_path)
    plt.close()

print("✅ 所有 PCA 图像已生成")

# 生成GIF
print("📽️ 正在生成动图...")
image_files = [os.path.join(output_dir, f"pca_epoch_{epoch:03d}.png") for epoch in epochs]
images = []
for f in tqdm(image_files, desc="Loading images for GIF"):
    images.append(Image.open(f))

images[0].save(gif_path, save_all=True, append_images=images[1:], duration=500, loop=0)

print(f"✅ 动图已保存为：{gif_path}")


## **3-用训练好的GNN模型预测新化合物的甜度(模型推理)**

### **单个预测**

In [ ]:
import torch
from rdkit import Chem
from torch_geometric.data import Data
import numpy as np

# 1. 确保设备正确
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 初始化模型
# ❗❗❗ 必须与训练时的超参数完全一致 ❗❗❗
# 根据上一轮训练代码 (Strategy B): hidden_dim=16, mlp_hidden=128, heads=2 (默认), edge_dim=6
model = GATNet(
    input_dim=69,        # 节点特征维度
    hidden_dim=16,       # 训练时用的 16
    mlp_hidden=128,      # 训练时用的 128
    output_dim=1,
    heads=2,             # 训练时默认是 2
    dropout=0.4,         # 保持一致 (虽然 eval 模式下不起作用)
    edge_dim=6           # ❗❗ 关键：必须指定边特征维度
).to(device)

# 3. 加载最佳模型权重
# 请确保文件名与刚才保存的一致 (Best_GATEdge+MLP_Model.pt)
model_path = "Best_GATEdge+MLP_Model.pt"
try:
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    print("✅ 模型权重加载成功")
except Exception as e:
    print(f"❌ 加载权重失败: {e}")
    print("请检查模型文件路径或超参数设置是否与训练时完全一致。")

# 辅助函数：SMILES转化为特征
def smiles_to_features(smiles, featurizer):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError(f"无法解析SMILES: {smiles}")

    features = featurizer(mol)

    # 转换为 Tensor
    x = torch.tensor(features['x'], dtype=torch.float32).to(device)
    edge_index = features['edge_index'].to(device)

    # 处理边特征
    if len(features['edge_attr']) > 0:
        edge_attr = torch.tensor(features['edge_attr'], dtype=torch.float32).to(device)
    else:
        # 如果没有边（如单个离子），生成空的 tensor
        edge_attr = torch.empty((0, 6), dtype=torch.float32).to(device)

    # ❗❗ 关键修改：生成 batch 向量 ❗❗
    # 对于单个分子预测，batch 是一个长度为节点数的全 0 向量
    # 这告诉 pool 层：由于这里只有一个图，所有节点都属于 index 0
    batch = torch.zeros(x.shape[0], dtype=torch.long).to(device)

    return x, edge_index, batch, edge_attr

# 主函数：预测甜度
def predict_sweetness(smiles, model, featurizer):
    try:
        # 获取数据
        x, edge_index, batch, edge_attr = smiles_to_features(smiles, featurizer)

        with torch.no_grad():
            # ❗❗ 关键：传入所有参数，包括 edge_attr ❗❗
            pred = model(x, edge_index, batch, edge_attr=edge_attr)

        return pred.item()
    except Exception as e:
        print(f"预测过程出错: {e}")
        return None

# 用户输入新的SMILES (Rebaudioside A, 甜菊糖苷A)
new_smiles = "CO[C@H]1[C@H](O[C@H]2CC[C@]3(C)[C@H]4CC[C@]5(C)[C@@H]([C@H](C)CCC[C@H](C)CO)[C@@H](O)[C@H](O)[C@H]5[C@]4(O)CC[C@H]3[C@H]2O)OC[C@@H](O)[C@@H]1O"

# 确保特征提取器已定义
if 'featurizer' not in locals():
    featurizer = MoleculeFeaturizer()

# 预测甜度
print(f"\n正在预测分子: {new_smiles[:30]}...")
predicted_sweetness = predict_sweetness(new_smiles, model, featurizer)

if predicted_sweetness is not None:
    print("="*40)
    print(f"预测结果 (logSw): {predicted_sweetness:.4f}")
    # 反推大概的相对甜度 (假设 logSw 是 log10(相对蔗糖甜度))
    # 注意：这取决于你原始数据的单位，如果 logSw 是 log10(SW)，那么：
    estimated_sw = 10 ** predicted_sweetness
    print(f"估算相对甜度 (SW): {estimated_sw:.2f}")
    print("="*40)

### **批量预测**

In [ ]:
import torch
import pandas as pd
from rdkit import Chem
from tqdm import tqdm  # 导入tqdm
import numpy as np

# 1. 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 初始化模型
# ❗❗❗ 必须与训练时的超参数完全一致 (Strategy B) ❗❗❗
model = GATNet(
    input_dim=69,        # 节点特征维度
    hidden_dim=16,       # 训练时用的 16
    mlp_hidden=128,      # 训练时用的 128
    output_dim=1,
    heads=2,             # 训练时默认是 2
    dropout=0.4,         # 保持一致
    edge_dim=6           # ❗❗ 关键：必须指定边特征维度
).to(device)

# 3. 加载最佳模型权重
model_path = "/content/Best_GATEdge+MLP_Model.pt"
try:
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    print("✅ 模型权重加载成功")
except RuntimeError as e:
    print(f"❌ 加载权重失败: {e}")
    print("请检查 GATNet 初始化参数是否与训练代码完全一致 (hidden_dim=16, mlp_hidden=128, heads=2).")
    exit()

# 辅助函数：SMILES转化为特征 Tensor
def smiles_to_features(smiles, featurizer):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None, None, None # 返回空以示错误

    features = featurizer(mol)

    # 转换为 Tensor 并移动到设备
    x = torch.tensor(features['x'], dtype=torch.float32).to(device)
    edge_index = features['edge_index'].to(device)

    # 处理边特征
    if len(features['edge_attr']) > 0:
        edge_attr = torch.tensor(features['edge_attr'], dtype=torch.float32).to(device)
    else:
        edge_attr = torch.empty((0, 6), dtype=torch.float32).to(device)

    # ❗❗ 关键修复：生成 batch 向量 ❗❗
    # 单个分子预测时，batch 是全 0 向量，长度等于原子数
    batch = torch.zeros(x.shape[0], dtype=torch.long).to(device)

    return x, edge_index, batch, edge_attr

# 主函数：预测甜度
def predict_sweetness(smiles, model, featurizer):
    try:
        x, edge_index, batch, edge_attr = smiles_to_features(smiles, featurizer)

        if x is None: # 解析失败
            return None

        with torch.no_grad():
            # ❗❗ 关键：传入所有参数，包括 edge_attr ❗❗
            pred = model(x, edge_index, batch, edge_attr=edge_attr)
        return pred.item()
    except Exception as e:
        # print(f"预测出错 ({smiles}): {e}") # 可选：打印具体错误
        return None

# ==========================================
# 批量处理 CSV 流程
# ==========================================

# 加载包含SMILES的CSV文件
csv_path = "/content/SweetpredDB.csv" # 确保文件路径正确

try:
    df = pd.read_csv(csv_path)
    print(f"成功读取 CSV，共 {len(df)} 行数据")
except FileNotFoundError:
    print(f"❌ 未找到文件: {csv_path}")
    exit()

# 确保列名正确 (有些数据集列名可能是 'SMILES' 或 'smiles')
if 'smiles' not in df.columns:
    # 尝试查找可能是 smiles 的列
    possible_cols = [col for col in df.columns if 'smiles' in col.lower()]
    if possible_cols:
        smiles_col = possible_cols[0]
        print(f"警告: 未找到 'smiles' 列，将使用 '{smiles_col}' 列代替")
    else:
        print("❌ CSV文件必须包含 'smiles' 列！")
        exit()
else:
    smiles_col = 'smiles'

# 确保特征提取器已定义
if 'MoleculeFeaturizer' not in locals():
    print("❌ 错误: MoleculeFeaturizer 未定义，请先运行定义该类的代码块。")
else:
    featurizer = MoleculeFeaturizer()

    # 用于保存预测结果
    predictions = []
    valid_indices = [] # 记录预测成功的行索引

    print("开始批量预测...")

    # 批量预测，并且添加进度条
    # 使用 tqdm 包装迭代器
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Predicting"):
        smiles = row[smiles_col]

        # 排除空值
        if pd.isna(smiles):
            predictions.append(None)
            continue

        pred_val = predict_sweetness(smiles, model, featurizer)
        predictions.append(pred_val)

    # 将预测结果添加到原始 DataFrame 中
    df['Predicted_logSw'] = predictions

    # 打印前几行查看结果
    print("\n预测完成！前 5 行结果：")
    print(df[[smiles_col, 'Predicted_logSw']].head())

    # 保存预测结果到CSV文件
    output_csv = "/content/prediction_result.csv"
    df.to_csv(output_csv, index=False)
    print(f"\n✅ 完整结果已保存到 {output_csv}")

## **4-基于梯度的GNN模型可解释性分析**

In [ ]:
!pip install matplotlib==3.7.3#安装后会显示重启，重启后，需要重新运行一遍所有前面的代码：（

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import SimilarityMaps, rdMolDraw2D
import numpy as np
import torch
from torch_geometric.loader import DataLoader
from tqdm import tqdm

# 1. 确保设备正确
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. 初始化模型 (参数保持 Strategy B)
model = GATNet(
    input_dim=69,
    hidden_dim=16,
    mlp_hidden=128,
    output_dim=1,
    heads=2,
    dropout=0.4,
    edge_dim=6
).to(device)

def visualize_grad_heatmap(model, val_dataset, device, num_samples=10):
    # 随机选择样本
    indices = np.random.choice(len(val_dataset), num_samples, replace=False)
    subset = torch.utils.data.Subset(val_dataset, indices)
    loader = DataLoader(subset, batch_size=1, shuffle=False)

    # 自定义颜色映射
    color_blue = (75 / 255, 116 / 255, 178 / 255)
    color_red = (219 / 255, 49 / 255, 36 / 255)
    custom_cmap = LinearSegmentedColormap.from_list(
        "custom_cmap", [color_blue, "white", color_red]
    )

    model.eval()
    print(f"正在生成 {num_samples} 张热力图...")

    for batch_idx, data in enumerate(tqdm(loader, desc="Generating heatmaps")):
        data = data.to(device)

        input_x = data.x.clone().detach().requires_grad_(True)
        pred = model(input_x, data.edge_index, data.batch, edge_attr=data.edge_attr)

        model.zero_grad()
        pred.backward()

        # 获取梯度
        grads = input_x.grad.abs().sum(dim=1).cpu().numpy()

        # 归一化
        if grads.max() - grads.min() > 0:
            grads = (grads - grads.min()) / (grads.max() - grads.min())

        # ❗❗❗ 关键修复 1：转换为 Python 列表 ❗❗❗
        # 这一步解决了 "The truth value of an array..." 错误
        weights_list = grads.tolist()

        smiles = data.smiles[0] if isinstance(data.smiles, list) else data.smiles
        mol = Chem.MolFromSmiles(smiles)

        if mol:
            try:
                AllChem.Compute2DCoords(mol)

                # 创建 RDKit 绘图对象
                d = rdMolDraw2D.MolDraw2DCairo(400, 400)

                # ❗❗❗ 关键修复 2：传入 weights_list 而不是 grads 数组 ❗❗❗
                SimilarityMaps.GetSimilarityMapFromWeights(
                    mol,
                    weights=weights_list,  # 这里传入 list
                    colorMap=custom_cmap,
                    contourLines=10,
                    draw2d=d
                )

                d.FinishDrawing()
                save_name = f"heatmap_{batch_idx}.png"
                d.WriteDrawingText(save_name)

            except Exception as e:
                print(f"绘制分子 {smiles[:10]}... 时出错: {e}")
                continue

    print(f"✅ 热力图生成完毕，图片已保存在当前目录下。")

# 使用示例
try:
    model.load_state_dict(torch.load("/content/Best_GATEdge+MLP_Model.pt", map_location=device))
    print("✅ 模型权重加载成功")

    if 'val_dataset' in locals():
        visualize_grad_heatmap(model, val_dataset, device, num_samples=5)
    else:
        print("❌ val_dataset 未定义")

except Exception as e:
    print(f"❌ 运行出错: {e}")

### **可视化梯度热力图解释**



在分子属性预测的任务中，尤其是通过图神经网络（GNN）进行学习时，我们常常希望理解模型是如何根据输入数据（例如分子的结构）进行决策的。梯度热力图是一个有效的工具，它帮助我们了解模型是如何“关注”分子中的不同部分，尤其是在预测过程中，哪些部分的特征对最终预测影响最大。接下来，我们将详细解释**梯度热力图**的工作原理，**为何要使用梯度热力图**，以及如何在本项目中结合图神经网络和化学知识生成热力图。

---

#### 一、梯度热力图的原理

**梯度热力图**通过计算模型输出对输入特征的梯度，来揭示模型对于每个特征的重要性。在本项目中，我们关注的是图神经网络（GATNet）在对分子进行预测时，哪些**原子特征**对最终预测（例如甜度预测）影响最大。

1. **梯度计算**：模型通过反向传播计算每个原子特征的梯度。梯度是指**模型输出（例如预测的甜度值）**对输入特征（如原子特征）的变化率。对于一个特定的分子，梯度值较大的原子说明模型认为这些原子对预测结果的贡献较大。
2. **归一化**：为了更好地可视化梯度，计算出的梯度值被归一化处理。归一化后，梯度值被缩放到[0, 1]的范围内，从而便于可视化。
3. **热力图绘制**：基于归一化后的梯度值，我们可以将其映射到分子的2D结构图中，形成一个**热力图**，通过颜色的变化来表示每个原子的重要性。

---

#### 二、模型中的梯度计算

本项目的目标是通过训练得到的**GATNet**模型预测分子的甜度（`logSw`）。在模型的**前向传播**过程中，GAT网络首先通过图卷积层（GATConv）提取原子和分子之间的关系信息，然后通过池化层和MLP头部进行预测。我们希望通过**梯度反向传播**来了解哪些原子特征对预测结果影响最大。

具体流程如下：
1. **前向传播**：模型根据输入的原子特征（`data.x`）和图结构（`data.edge_index`），计算预测值（`pred`）。
2. **反向传播**：在预测完成后，我们计算**损失函数的梯度**。梯度反映了模型输出（预测值）对输入特征（原子特征）的灵敏度，即哪些原子对预测结果贡献更大。
3. **梯度的归一化**：为了便于可视化，梯度值被归一化到[0, 1]的范围内。归一化有助于将不同分子的梯度进行统一的尺度比较，使得我们可以直接观察哪些原子对于预测结果影响较大。

---

#### 三、结合分子可视化绘制热力图

在进行梯度计算后，我们需要将其与分子的结构进行对比。具体来说，我们将梯度值映射到**分子的二维结构图**中，并使用颜色来表示每个原子的贡献。

1. **分子结构绘制**：通过使用`RDKit`库的`Compute2DCoords`函数，生成每个分子的二维坐标。这样，分子就可以在平面上可视化。
2. **热力图生成**：通过`SimilarityMaps.GetSimilarityMapFromWeights`函数，我们将归一化后的梯度值映射到分子结构的每个原子上。热力图中的颜色从**白色**（表示较小的梯度）到**红色**（表示较大的梯度），通过这种方式直观地表示了每个原子在最终预测中的重要性。

在热力图中：
- **红色**区域表示该位置的原子对于预测结果有较大的贡献。
- **白色**区域表示该位置的原子对预测结果的贡献较小。

这种可视化不仅帮助我们理解模型是如何做出预测的，还能揭示分子结构与其预测性质之间的关系。

---

#### 四、模型与化学知识的结合

- **图神经网络与分子结构**：在分子数据中，原子和它们之间的键可以通过图的节点和边来表示。GATNet作为图神经网络的一种，通过图卷积层（GATConv）学习原子节点之间的关系，并在此基础上进行预测。在该模型中，边特征（如化学键类型、芳香性等）对预测结果的影响被有效地学习。
  
- **梯度热力图与化学解释**：通过梯度热力图，我们可以将分子特征和模型输出之间的关系可视化。例如，在预测分子的甜度时，模型可能会特别关注某些重要的化学基团（如羟基、烯烃基团等），这些基团的结构对分子的甜度有显著影响。通过热力图，我们可以识别这些关键区域，从而为分子设计提供化学指导。

- **分子设计与优化**：通过分析梯度热力图，我们不仅能够解释模型的预测行为，还能够**指导分子设计**。例如，如果某些特定的原子位置对甜度预测有较大影响，那么可以通过优化这些位置的化学基团来改进分子的甜度特性。

---

#### 五、总结

通过梯度热力图的可视化，我们可以深入了解模型在进行甜度预测时如何关注分子结构的不同部分。具体而言：
- **梯度热力图**通过计算每个原子对预测结果的贡献，帮助我们理解哪些部分的分子结构对最终预测最为关键。
- 结合**图神经网络**（GATNet）的图卷积操作，我们能够学习到分子之间复杂的结构关系。
- 使用**RDKit**进行分子可视化，将梯度信息映射到分子的二维结构图上，使得我们能够直观地看到高贡献区域，进而为分子设计和优化提供指导。

这种可视化方法不仅提升了我们对模型行为的理解，还为化学领域的分子优化和设计提供了重要的工具。

## **5-刚才是回归任务，我们再试试分类任务吧**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv, global_mean_pool
from rdkit import Chem
from rdkit.Chem import rdchem
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random
import os
from scipy.sparse import coo_matrix

# ==========================================
# 1. 配置与重置
# ==========================================
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 绘图设置
plt.rcParams['figure.dpi'] = 150
plt.style.use('seaborn-v0_8-whitegrid')

# ==========================================
# 2. 特征提取器 (保持不变，用于生成图数据)
# ==========================================
def one_hot_encoding(value, choices):
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1
    return encoding

class MoleculeFeaturizer(object):
    def __init__(self):
        pass

    def _atom_featurizer(self, atom):
        # 37种元素 + 1 other
        atomic_numer = [1, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 28, 29,
                        30, 31, 32, 33, 34, 35, 36, 37, 38, 46, 47, 48, 49, 50, 51, 52, 53]
        atom_type = one_hot_encoding(atom.GetAtomicNum(), atomic_numer)

        degree = one_hot_encoding(atom.GetTotalDegree(), list(range(5)))
        hybrid = one_hot_encoding(int(atom.GetHybridization()), list(range(len(Chem.HybridizationType.names)-1)))
        chiral = one_hot_encoding(atom.GetChiralTag(), list(range(len(Chem.ChiralType.names)-1)))
        num_hs = one_hot_encoding(atom.GetTotalNumHs(), list(range(5)))
        aromatic = [1 if atom.GetIsAromatic() else 0]

        return atom_type + degree + hybrid + chiral + num_hs + aromatic

    def _bond_featurizer(self, bond):
        bond_type = bond.GetBondType()
        bt = [
            int(bond_type == rdchem.BondType.SINGLE),
            int(bond_type == rdchem.BondType.DOUBLE),
            int(bond_type == rdchem.BondType.TRIPLE),
            int(bond_type == rdchem.BondType.AROMATIC)
        ]
        conj = [int(bond.GetIsConjugated()), int(bond.IsInRing())]
        return bt + conj

    def __call__(self, mol):
        # 节点特征
        atom_features = [self._atom_featurizer(atom) for atom in mol.GetAtoms()]
        x = torch.tensor(atom_features, dtype=torch.float32)

        # 边特征与索引
        adj = Chem.GetAdjacencyMatrix(mol)
        coo_adj = coo_matrix(adj)
        row, col = coo_adj.row, coo_adj.col
        edge_index = torch.tensor([row, col], dtype=torch.long)

        bond_features = []
        for i, j in zip(row, col):
            bond = mol.GetBondBetweenAtoms(int(i), int(j))
            bond_features.append(self._bond_featurizer(bond))

        if len(bond_features) > 0:
            edge_attr = torch.tensor(bond_features, dtype=torch.float32)
        else:
            edge_attr = torch.empty((0, 6), dtype=torch.float32)

        return {'x': x, 'edge_index': edge_index, 'edge_attr': edge_attr}

# ==========================================
# 3. 数据集处理 (转为分类任务)
# ==========================================
class MolClassificationDataset:
    def __init__(self, csv_path, threshold=3.0):
        self.csv_path = csv_path
        self.threshold = threshold # 阈值：>3 为高甜(1)，<=3 为低甜(0)
        self.featurizer = MoleculeFeaturizer()
        self.data_list = []
        self._process()

    def _process(self):
        df = pd.read_csv(self.csv_path)
        print(f"原始数据量: {len(df)}")

        valid_cnt = 0
        high_cnt = 0

        for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing"):
            mol = Chem.MolFromSmiles(row['Smiles'])
            if mol is None: continue

            try:
                feats = self.featurizer(mol)
                data = Data(x=feats['x'], edge_index=feats['edge_index'], edge_attr=feats['edge_attr'])

                # ❗ 关键：转换为分类标签 ❗
                raw_val = row['logSw']
                label = 1 if raw_val > self.threshold else 0

                if label == 1: high_cnt += 1

                data.y = torch.tensor([label], dtype=torch.long) # 分类任务用 Long 类型
                data.smiles = row['Smiles']

                self.data_list.append(data)
                valid_cnt += 1
            except:
                continue

        print(f"处理完成。有效样本: {valid_cnt}")
        print(f"类别分布 -> 高甜(1): {high_cnt}, 低甜(0): {valid_cnt - high_cnt}")

    def __len__(self): return len(self.data_list)
    def __getitem__(self, idx): return self.data_list[idx]

# 加载数据
dataset_cls = MolClassificationDataset(csv_path='/content/SweetpredDB.csv', threshold=3.0)

# 划分数据集 (8:1:1)
train_size = int(0.8 * len(dataset_cls))
val_size = int(0.1 * len(dataset_cls))
test_size = len(dataset_cls) - train_size - val_size
train_ds, val_ds, test_ds = torch.utils.data.random_split(dataset_cls, [train_size, val_size, test_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)
test_loader = DataLoader(test_ds, batch_size=32)

# ==========================================
# 4. GAT 分类模型定义
# ==========================================
class GATClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, heads=2, edge_dim=6, dropout=0.3):
        super().__init__()

        # GAT 层 (带边特征)
        self.gat1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=dropout, edge_dim=edge_dim)
        self.gat2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, dropout=dropout, edge_dim=edge_dim)

        # 分类 MLP 头
        # 二分类任务：输出维度可以是 2 (CrossEntropy) 或 1 (BCEWithLogits)
        # 这里使用 2，对应 softmax 输出两个类别的概率
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, 2)  # 输出 [logits_class_0, logits_class_1]
        )

    def forward(self, x, edge_index, batch, edge_attr):
        x = self.gat1(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)
        x = self.gat2(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)

        # 全局池化
        x = global_mean_pool(x, batch)

        # MLP 分类
        logits = self.mlp(x)
        return logits

# 初始化模型
input_dim = dataset_cls[0].x.shape[1] # 69
model = GATClassifier(input_dim=input_dim, hidden_dim=16, heads=2, edge_dim=6, dropout=0.4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-3)
criterion = nn.CrossEntropyLoss() # 自动处理 softmax

# ==========================================
# 5. 训练与评估流程
# ==========================================
def train_epoch():
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch, batch.edge_attr)
        loss = criterion(out, batch.y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * batch.num_graphs
        pred = out.argmax(dim=1)
        correct += (pred == batch.y).sum().item()
        total += batch.num_graphs
    return total_loss / total, correct / total

def eval_epoch(loader):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    preds_all, labels_all, probs_all = [], [], []

    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index, batch.batch, batch.edge_attr)
            loss = criterion(out, batch.y)

            total_loss += loss.item() * batch.num_graphs
            pred = out.argmax(dim=1)
            prob = F.softmax(out, dim=1)[:, 1] # 获取“高甜”类的概率

            correct += (pred == batch.y).sum().item()
            total += batch.num_graphs

            preds_all.extend(pred.cpu().numpy())
            labels_all.extend(batch.y.cpu().numpy())
            probs_all.extend(prob.cpu().numpy())

    return total_loss / total, correct / total, labels_all, preds_all, probs_all

# --- 开始训练 ---
epochs = 100
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
best_val_acc = 0

print("🚀 开始分类任务训练...")

for epoch in range(1, epochs+1):
    t_loss, t_acc = train_epoch()
    v_loss, v_acc, _, _, _ = eval_epoch(val_loader)

    history['train_loss'].append(t_loss)
    history['train_acc'].append(t_acc)
    history['val_loss'].append(v_loss)
    history['val_acc'].append(v_acc)

    if v_acc > best_val_acc:
        best_val_acc = v_acc
        torch.save(model.state_dict(), "Best_GAT_Classifier.pt")

    if epoch % 10 == 0:
        print(f"Epoch {epoch:03d} | Train Loss: {t_loss:.4f} Acc: {t_acc:.4f} | Val Loss: {v_loss:.4f} Acc: {v_acc:.4f}")

# ==========================================
# 6. 可视化评估结果
# ==========================================
# 加载最佳模型并在测试集上评估
model.load_state_dict(torch.load("Best_GAT_Classifier.pt"))
test_loss, test_acc, y_true, y_pred, y_prob = eval_epoch(test_loader)

print("\n" + "="*30)
print(f"最终测试集准确率: {test_acc:.4f}")
print("="*30)
print("\n分类报告:")
print(classification_report(y_true, y_pred, target_names=['Low Sweetness', 'High Sweetness']))

# --- 绘图 ---
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. 损失曲线
axes[0,0].plot(history['train_loss'], label='Train Loss', color='#4c72b0')
axes[0,0].plot(history['val_loss'], label='Val Loss', color='#c44e52', linestyle='--')
axes[0,0].set_title('Loss Curve')
axes[0,0].legend()

# 2. 准确率曲线
axes[0,1].plot(history['train_acc'], label='Train Acc', color='#55a868')
axes[0,1].plot(history['val_acc'], label='Val Acc', color='#8172b3', linestyle='--')
axes[0,1].set_title('Accuracy Curve')
axes[0,1].legend()

# 3. 混淆矩阵
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1,0],
            xticklabels=['Low', 'High'], yticklabels=['Low', 'High'])
axes[1,0].set_xlabel('Predicted')
axes[1,0].set_ylabel('Actual')
axes[1,0].set_title('Confusion Matrix (Test Set)')

# 4. ROC 曲线
fpr, tpr, _ = roc_curve(y_true, y_prob)
roc_auc = auc(fpr, tpr)
axes[1,1].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
axes[1,1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[1,1].set_xlim([0.0, 1.0])
axes[1,1].set_ylim([0.0, 1.05])
axes[1,1].set_xlabel('False Positive Rate')
axes[1,1].set_ylabel('True Positive Rate')
axes[1,1].set_title('ROC Curve')
axes[1,1].legend(loc="lower right")

plt.tight_layout()
plt.show()

这段代码展示了一个基于图注意力网络（GAT）的**二分类模型**，用于预测分子是“高甜”还是“低甜”。

以下是针对代码逻辑、分类实现细节以及与回归任务区别的详细解释。


#### 1\. GATClassifier 模型架构：如何实现分类？

这个模型的核心结构与回归模型非常相似，**最关键的区别在于最后一层 MLP 的输出维度**。

  * **回归任务 (Regression)**：

      * 目标是预测一个连续的实数值（如甜度值 `logSw`）。
      * 输出层通常是 `nn.Linear(hidden_dim, 1)`。
      * 输出结果直接就是预测值（例如 3.45）。

  * **分类任务 (Classification)**：

      * 目标是预测样本属于哪一个类别（0: 低甜, 1: 高甜）。
      * **代码实现**：
        ```python
        nn.Linear(32, 2)  # 输出维度为 2
        ```
      * **含义**：输出一个形状为 `[Batch_Size, 2]` 的张量。对于每一个分子，模型输出两个分数（Logits）：
          * 第 0 个数字：分子属于“低甜”类别的得分。
          * 第 1 个数字：分子属于“高甜”类别的得分。

-----

#### 2\. `criterion` 是什么？

在 PyTorch 的分类任务训练循环中，`criterion` 通常代表**损失函数 (Loss Function)**。

  * **在本代码的上下文中**：`criterion` 指的是 **`nn.CrossEntropyLoss()`**（交叉熵损失）。
  * **它的作用**：
    1.  它接收模型的原始输出 `out`（包含两个分数的 Logits）。
    2.  它接收真实的标签 `batch.y`（0 或 1）。
    3.  它内部会自动先对 `out` 做 Softmax 操作，计算出概率分布，然后计算预测概率分布与真实标签之间的距离（损失）。
  * **与回归的区别**：回归任务通常使用 `nn.MSELoss()`（均方误差损失）。

-----

#### 3\. `out.argmax(dim=1)` 是什么意思？

这行代码用于**获取最终的预测类别标签**（硬分类）。

  * **`out` 的数据样子**：假设一个 Batch 有 3 个分子，模型输出如下：
    ```text
    [[2.1, 0.5],   # 分子1: 低甜分更高 -> 预测 0
     [-1.0, 3.2],  # 分子2: 高甜分更高 -> 预测 1
     [0.8, 0.9]]   # 分子3: 高甜分稍高 -> 预测 1
    ```
  * **`argmax(dim=1)`**：
      * `dim=1` 表示在“列”的方向（类别分数）上进行比较。
      * 它会返回**最大值的索引**。
  * **结果**：
      * 分子1：2.1 \> 0.5，最大值索引是 **0**。
      * 分子2：3.2 \> -1.0，最大值索引是 **1**。
      * 分子3：0.9 \> 0.8，最大值索引是 **1**。
      * 最终得到 `pred = [0, 1, 1]`。

-----

#### 4\. `prob = F.softmax(out, dim=1)[:, 1]` 是什么意思？

这行代码用于**获取分子属于“高甜”类别的概率**（软分类），通常用于绘制 ROC 曲线或计算 AUC。

1.  **`F.softmax(out, dim=1)`**：

      * 模型的原始输出（Logits）范围是 $(-\infty, +\infty)$，不直观。
      * Softmax 将这些分数压缩到 $(0, 1)$ 之间，并且两个类别的概率之和为 1。
      * *示例变换*：`[2.1, 0.5]` -\> Softmax -\> `[0.83, 0.17]`（即 83% 是低甜，17% 是高甜）。

2.  **`[:, 1]`**：

      * 这是一个切片操作。
      * `:` 表示取所有样本（行）。
      * `1` 表示只取第 1 列的数据（即**类别 1 / 高甜**的概率）。
      * *结果*：我们只关心模型认为它是“高甜”的概率是多少。

-----

#### 5\. 总结：分类 vs 回归 的代码差异对照表

| 特性 | 回归任务 (Regression) | 分类任务 (Classification) |
| :--- | :--- | :--- |
| **MLP 输出层** | `nn.Linear(..., 1)` | `nn.Linear(..., 2)` (二分类) |
| **输出含义** | 具体的数值 (如甜度值 4.5) | 属于各类别的“得分” (Logits) |
| **标签数据类型** | `torch.float32` (浮点数) | `torch.long` (整数索引 0, 1) |
| **损失函数** | `MSELoss` / `L1Loss` | `CrossEntropyLoss` |
| **获取预测结果** | 直接取输出值 `out` | `out.argmax(dim=1)` (取分数最高的类别) |
| **评估指标** | MSE, RMSE, R² | Accuracy, AUC, F1-score, Confusion Matrix |

# **小分子任务——基于GNN的药物药物相互作用预测-图级别任务**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv, global_mean_pool
from rdkit import Chem
from rdkit.Chem import rdchem
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.sparse import coo_matrix
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    average_precision_score,
    roc_curve,
    precision_recall_curve,
)
import matplotlib.pyplot as plt
import os

# ==========================================
# 1. 分子特征提取器 (Featurizer)
# ==========================================
# 辅助函数：One-hot编码
def one_hot_encoding(value, choices):
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1
    return encoding

class MoleculeFeaturizer(object):
    """
    将RDKit分子对象转换为PyG图数据 (Data对象)
    包含节点特征(69维) 和 边特征(6维)
    """
    def __init__(self):
        pass

    def _atom_featurizer(self, atom):
        # 37种原子类型 + 1种其他
        atomic_numer = [1, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 28, 29,
                        30, 31, 32, 33, 34, 35, 36, 37, 38, 46, 47, 48, 49, 50, 51, 52, 53]
        atom_type = one_hot_encoding(atom.GetAtomicNum(), atomic_numer)

        degree = one_hot_encoding(atom.GetTotalDegree(), list(range(5)))
        hybrid = one_hot_encoding(int(atom.GetHybridization()), list(range(len(Chem.HybridizationType.names)-1)))
        chiral = one_hot_encoding(atom.GetChiralTag(), list(range(len(Chem.ChiralType.names)-1)))
        num_hs = one_hot_encoding(atom.GetTotalNumHs(), list(range(5)))
        aromatic = [1 if atom.GetIsAromatic() else 0]

        return atom_type + degree + hybrid + chiral + num_hs + aromatic

    def _bond_featurizer(self, bond):
        bond_type = bond.GetBondType()
        bt = [
            int(bond_type == rdchem.BondType.SINGLE),
            int(bond_type == rdchem.BondType.DOUBLE),
            int(bond_type == rdchem.BondType.TRIPLE),
            int(bond_type == rdchem.BondType.AROMATIC)
        ]
        conj = [int(bond.GetIsConjugated()), int(bond.IsInRing())]
        return bt + conj

    def __call__(self, mol):
        # 1. 节点特征 (Node Features)
        atom_features = [self._atom_featurizer(atom) for atom in mol.GetAtoms()]
        x = torch.tensor(atom_features, dtype=torch.float32)

        # 2. 边索引与边特征 (Edge Index & Attributes)
        adj = Chem.GetAdjacencyMatrix(mol)
        coo_adj = coo_matrix(adj)
        row, col = coo_adj.row, coo_adj.col
        edge_index = torch.tensor([row, col], dtype=torch.long)

        bond_features = []
        for i, j in zip(row, col):
            bond = mol.GetBondBetweenAtoms(int(i), int(j))
            bond_features.append(self._bond_featurizer(bond))

        if len(bond_features) > 0:
            edge_attr = torch.tensor(bond_features, dtype=torch.float32)
        else:
            # 如果没有边，提供空的占位符
            edge_attr = torch.empty((0, 6), dtype=torch.float32)

        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

# ==========================================
# 2. DDI 数据集与 DataLoader
# ==========================================
class DDIDataset:
    def __init__(self, csv_path, sample_ratio=0.1, seed=42):
        self.data_list = []
        self.featurizer = MoleculeFeaturizer()

        print(f"正在加载并处理数据: {csv_path} ...")
        df = pd.read_csv(csv_path)

        # ⭐⭐⭐ 只随机选取部分数据
        df = df.sample(frac=sample_ratio, random_state=seed).reset_index(drop=True)
        print(f"⚠️ 已抽取 {sample_ratio*100:.0f}% 样本用于训练，共 {len(df)} 条数据")

        for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing DDI Pairs"):
            s1, s2, label = row['smiles_1'], row['smiles_2'], row['label']

            mol1 = Chem.MolFromSmiles(s1)
            mol2 = Chem.MolFromSmiles(s2)

            if mol1 is None or mol2 is None:
                continue

            data1 = self.featurizer(mol1)
            data2 = self.featurizer(mol2)
            y = torch.tensor([label], dtype=torch.float32)

            self.data_list.append((data1, data2, y))

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

# 自定义 Collate 函数：用于将成对的图打包成 Batch
def collate_ddi_pairs(batch):
    data1_list = [item[0] for item in batch]
    data2_list = [item[1] for item in batch]
    labels = [item[2] for item in batch]

    # 将图列表转换为PyG的大图Batch
    batch1 = Batch.from_data_list(data1_list)
    batch2 = Batch.from_data_list(data2_list)
    labels = torch.stack(labels)

    return batch1, batch2, labels

# ==========================================
# 3. GAT Siamese 模型架构
# ==========================================
class GATEncoder(nn.Module):
    """共享权重的图编码器"""
    def __init__(self, input_dim, hidden_dim, heads=2, edge_dim=6, dropout=0.2):
        super().__init__()
        # 第一层 GAT
        self.gat1 = GATConv(input_dim, hidden_dim, heads=heads, dropout=dropout, edge_dim=edge_dim)
        # 第二层 GAT
        self.gat2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, dropout=dropout, edge_dim=edge_dim)

    def forward(self, x, edge_index, batch, edge_attr):
        x = self.gat1(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)
        x = self.gat2(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)
        # 全局池化得到图嵌入
        x = global_mean_pool(x, batch)
        return x

class DDIPredictor(nn.Module):
    """DDI 预测主模型"""
    def __init__(self, input_dim=69, hidden_dim=64, heads=2, edge_dim=6):
        super().__init__()
        # 编码器 (用于提取分子特征)
        self.encoder = GATEncoder(input_dim, hidden_dim, heads, edge_dim)

        # 预测头 (MLP)
        # 输入维度是 hidden_dim * 2，因为拼接了两个药物的特征
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim * 2, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 1)  # 二分类输出一个 Logit
        )

    def forward(self, batch1, batch2):
        # 分别编码两个药物
        emb1 = self.encoder(batch1.x, batch1.edge_index, batch1.batch, batch1.edge_attr)
        emb2 = self.encoder(batch2.x, batch2.edge_index, batch2.batch, batch2.edge_attr)

        # 拼接特征
        combined = torch.cat([emb1, emb2], dim=1)

        # 预测
        logits = self.mlp(combined)
        return logits

# ==========================================
# 4. 评估函数
# ==========================================
def evaluate(model, loader, device, return_details=False):
    """
    return_details=False: 只返回 acc, auc
    return_details=True:  返回 acc, auc, y_true, y_scores
    """
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for b1, b2, y in loader:
            b1, b2, y = b1.to(device), b2.to(device), y.to(device)
            logits = model(b1, b2)
            probs = torch.sigmoid(logits)  # [B,1]

            # 展平后再收集，防止多一维
            all_preds.extend(probs.cpu().view(-1).numpy())
            all_labels.extend(y.cpu().view(-1).numpy())

    # 计算指标
    y_true = np.array(all_labels)
    y_scores = np.array(all_preds)
    y_pred_cls = (y_scores > 0.5).astype(int)

    acc = accuracy_score(y_true, y_pred_cls)
    try:
        auc = roc_auc_score(y_true, y_scores)
    except:
        auc = 0.5  # 防止单类别报错

    if return_details:
        return acc, auc, y_true, y_scores
    else:
        return acc, auc

# ==========================================
# 5. 训练与绘图脚本
# ==========================================
def train_ddi_model():
    # 设置设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # 1. 加载数据集
    dataset = DDIDataset('DDI_balanced.csv')

    # 2. 划分训练集和测试集 (8:2)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_ds, test_ds = torch.utils.data.random_split(dataset, [train_size, test_size])

    # 3. 创建 DataLoader (注意使用自定义的 collate_fn)
    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, collate_fn=collate_ddi_pairs)
    test_loader = DataLoader(test_ds, batch_size=64, shuffle=False, collate_fn=collate_ddi_pairs)

    # 4. 初始化模型
    model = DDIPredictor(input_dim=69, hidden_dim=64).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.BCEWithLogitsLoss()  # 结合了 Sigmoid 和 BCELoss，数值更稳定

    # 5. 训练循环
    epochs = 20
    print("开始训练...")

    train_losses = []
    test_accs = []
    test_aucs = []

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0

        for b1, b2, y in tqdm(train_loader, desc=f"Epoch {epoch}/{epochs}"):
            b1, b2, y = b1.to(device), b2.to(device), y.to(device)

            optimizer.zero_grad()
            logits = model(b1, b2)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        train_losses.append(avg_loss)

        # 每个 Epoch 进行评估
        acc, auc = evaluate(model, test_loader, device)
        test_accs.append(acc)
        test_aucs.append(auc)

        print(f"Epoch {epoch}: Train Loss = {avg_loss:.4f}, Test Acc = {acc:.4f}, Test AUC = {auc:.4f}")

    # 6. 训练完成后再做一次详细评估（用于画ROC/PR曲线）
    final_acc, final_auc, y_true, y_scores = evaluate(model, test_loader, device, return_details=True)
    ap = average_precision_score(y_true, y_scores)
    print(f"Final Test Acc = {final_acc:.4f}, AUC = {final_auc:.4f}, AP = {ap:.4f}")

    # 7. 绘制训练过程曲线
    epochs_range = range(1, epochs + 1)

    # (1) Loss 曲线
    plt.figure()
    plt.plot(epochs_range, train_losses, marker='o')
    plt.xlabel("Epoch")
    plt.ylabel("Train Loss")
    plt.title("Training Loss Curve")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("loss_curve.png", dpi=300)
    plt.show()

    # (2) 测试集 Acc & AUC 曲线
    plt.figure()
    plt.plot(epochs_range, test_accs, marker='o', label="Test Accuracy")
    plt.plot(epochs_range, test_aucs, marker='s', label="Test AUC")
    plt.xlabel("Epoch")
    plt.ylabel("Metric Value")
    plt.title("Test Accuracy & AUC over Epochs")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("metrics_curve.png", dpi=300)
    plt.show()

    # (3) ROC & PR 曲线
    fpr, tpr, _ = roc_curve(y_true, y_scores)
    precision, recall, _ = precision_recall_curve(y_true, y_scores)

    plt.figure()
    plt.plot(fpr, tpr, label=f"ROC (AUC={final_auc:.3f})")
    plt.plot([0, 1], [0, 1], linestyle="--", label="Random")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("roc_curve.png", dpi=300)
    plt.show()

    plt.figure()
    plt.plot(recall, precision, label=f"PR (AP={ap:.3f})")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall Curve")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("pr_curve.png", dpi=300)
    plt.show()

    # 8. 保存模型
    torch.save(model.state_dict(), "DDI_GAT_Model.pt")
    print("模型已保存为 DDI_GAT_Model.pt")
    print("Loss 曲线已保存为 loss_curve.png")
    print("Acc/AUC 曲线已保存为 metrics_curve.png")
    print("ROC 曲线已保存为 roc_curve.png")
    print("PR 曲线已保存为 pr_curve.png")

if __name__ == "__main__":
    # 确保当前目录下有 'DDI_balanced.csv'
    if os.path.exists('DDI_balanced.csv'):
        train_ddi_model()
    else:
        print("未找到 DDI_balanced.csv 文件，请先运行数据平衡代码。")


# **小分子任务——基于GNN的R/S E/Z预测-节点和边级别任务**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv
from rdkit import Chem
from rdkit.Chem import rdchem
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random
from scipy.sparse import coo_matrix

# 1. 基础配置
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
plt.rcParams['figure.dpi'] = 100

# 2. 辅助函数
def one_hot_encoding(value, choices):
    encoding = [0] * (len(choices) + 1)
    index = choices.index(value) if value in choices else -1
    encoding[index] = 1
    return encoding

# 3. 特征提取器
class RS_Featurizer(object):
    def __call__(self, mol):
        labels, mask = [], []
        chiral_centers = Chem.FindMolChiralCenters(mol, includeUnassigned=False)
        center_dict = {idx: type for idx, type in chiral_centers}

        for atom in mol.GetAtoms():
            idx = atom.GetIdx()
            if idx in center_dict:
                stereo = center_dict[idx]
                if stereo == 'R': labels.append(0); mask.append(1)
                elif stereo == 'S': labels.append(1); mask.append(1)
                else: labels.append(2); mask.append(0)
            else:
                labels.append(2); mask.append(0)

        atom_features = []
        for atom in mol.GetAtoms():
            f = one_hot_encoding(atom.GetAtomicNum(), [1, 5, 6, 7, 8, 9, 15, 16, 17, 35, 53]) + \
                one_hot_encoding(atom.GetTotalDegree(), [0, 1, 2, 3, 4]) + \
                one_hot_encoding(int(atom.GetHybridization()), [2, 3, 4]) + \
                one_hot_encoding(atom.GetTotalNumHs(), [0, 1, 2, 3, 4])
            atom_features.append(f)

        x = torch.tensor(atom_features, dtype=torch.float32)
        y = torch.tensor(labels, dtype=torch.long)
        train_mask = torch.tensor(mask, dtype=torch.bool)

        adj = Chem.GetAdjacencyMatrix(mol)
        coo_adj = coo_matrix(adj)
        edge_index = torch.tensor([coo_adj.row, coo_adj.col], dtype=torch.long)

        bond_feats = []
        for i, j in zip(coo_adj.row, coo_adj.col):
            bond = mol.GetBondBetweenAtoms(int(i), int(j))
            bt = bond.GetBondType()
            f = [int(bt == t) for t in [rdchem.BondType.SINGLE, rdchem.BondType.DOUBLE,
                                        rdchem.BondType.TRIPLE, rdchem.BondType.AROMATIC]]
            bond_feats.append(f)
        edge_attr = torch.tensor(bond_feats, dtype=torch.float32)

        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y, train_mask=train_mask)

class EZ_Featurizer(object):
    def __call__(self, mol):
        atom_features = []
        for atom in mol.GetAtoms():
             f = one_hot_encoding(atom.GetAtomicNum(), [1, 6, 7, 8, 9, 15, 16, 17]) + \
                 one_hot_encoding(atom.GetTotalDegree(), [0, 1, 2, 3])
             atom_features.append(f)
        x = torch.tensor(atom_features, dtype=torch.float32)

        adj = Chem.GetAdjacencyMatrix(mol)
        coo_adj = coo_matrix(adj)
        edge_index = torch.tensor([coo_adj.row, coo_adj.col], dtype=torch.long)

        bond_feats, labels, mask = [], [], []

        for i, j in zip(coo_adj.row, coo_adj.col):
            bond = mol.GetBondBetweenAtoms(int(i), int(j))
            bt = bond.GetBondType()
            f = [int(bt == t) for t in [rdchem.BondType.SINGLE, rdchem.BondType.DOUBLE,
                                        rdchem.BondType.TRIPLE, rdchem.BondType.AROMATIC]]
            bond_feats.append(f)

            stereo = bond.GetStereo()
            if stereo == rdchem.BondStereo.STEREOE: labels.append(0); mask.append(1)
            elif stereo == rdchem.BondStereo.STEREOZ: labels.append(1); mask.append(1)
            else: labels.append(2); mask.append(0)

        edge_attr = torch.tensor(bond_feats, dtype=torch.float32)
        y = torch.tensor(labels, dtype=torch.long)
        train_mask = torch.tensor(mask, dtype=torch.bool)

        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y, train_mask=train_mask)

class MultiTaskDataset:
    def __init__(self, csv_path, mode='RS'):
        self.data_list = []
        if mode == 'RS': self.featurizer = RS_Featurizer()
        else: self.featurizer = EZ_Featurizer()
        df = pd.read_csv(csv_path)
        for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {mode}"):
            mol = Chem.MolFromSmiles(row['Smiles'])
            if mol is None: continue
            data = self.featurizer(mol)
            if data.train_mask.sum() > 0:
                self.data_list.append(data)
    def __len__(self): return len(self.data_list)
    def __getitem__(self, idx): return self.data_list[idx]

# 4. 模型
class GAT_TaskModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, edge_dim, task_type='node'):
        super().__init__()
        self.task_type = task_type
        self.gat1 = GATConv(input_dim, hidden_dim, heads=2, edge_dim=edge_dim)
        self.gat2 = GATConv(hidden_dim * 2, hidden_dim, heads=1, edge_dim=edge_dim)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim if task_type=='node' else hidden_dim*2, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )

    def forward(self, x, edge_index, edge_attr):
        x = self.gat1(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)
        x = self.gat2(x, edge_index, edge_attr=edge_attr)
        x = F.elu(x)

        if self.task_type == 'node':
            return self.classifier(x)
        elif self.task_type == 'edge':
            row, col = edge_index
            edge_features = torch.cat([x[row], x[col]], dim=1)
            return self.classifier(edge_features)

# 5. 训练与评估
def train_task(loader, model, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.edge_attr)
        mask = batch.train_mask
        if mask.sum() == 0: continue
        loss = criterion(out[mask], batch.y[mask])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def eval_task_probs(loader, model):
    model.eval()
    y_true, y_probs = [], []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index, batch.edge_attr)
            mask = batch.train_mask
            if mask.sum() == 0: continue

            # Get probabilities for class 1
            probs = F.softmax(out[mask], dim=1)[:, 1]

            y_true.extend(batch.y[mask].cpu().numpy())
            y_probs.extend(probs.cpu().numpy())
    return np.array(y_true), np.array(y_probs)

def run_task(task_name, dataset, task_type, edge_dim):
    print(f"\nRunning {task_name} Task...")
    if len(dataset) == 0:
        print("No data found.")
        return

    train_data, test_data = train_test_split(dataset.data_list, test_size=0.2, random_state=42)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32)

    model = GAT_TaskModel(input_dim=dataset[0].x.shape[1], hidden_dim=32,
                          edge_dim=edge_dim, task_type=task_type).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    criterion = nn.CrossEntropyLoss()

    train_losses = []
    for epoch in range(1, 21):
        loss = train_task(train_loader, model, optimizer, criterion)
        train_losses.append(loss)
        if epoch % 10 == 0: print(f"Epoch {epoch}: Loss {loss:.4f}")

    y_true, y_probs = eval_task_probs(test_loader, model)
    y_pred = (y_probs > 0.5).astype(int)

    # Plotting
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # 1. Loss Curve
    axes[0].plot(train_losses, label='Train Loss')
    axes[0].set_title(f'{task_name} - Loss Curve')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()

    # 2. Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1])
    axes[1].set_title(f'{task_name} - Confusion Matrix')
    axes[1].set_xlabel('Predicted')
    axes[1].set_ylabel('True')

    # 3. ROC Curve
    fpr, tpr, _ = roc_curve(y_true, y_probs)
    roc_auc = auc(fpr, tpr)
    axes[2].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    axes[2].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    axes[2].set_xlim([0.0, 1.0])
    axes[2].set_ylim([0.0, 1.05])
    axes[2].set_xlabel('False Positive Rate')
    axes[2].set_ylabel('True Positive Rate')
    axes[2].set_title(f'{task_name} - ROC Curve')
    axes[2].legend(loc="lower right")

    plt.tight_layout()
    plt.show()

    print(f"{task_name} Accuracy: {accuracy_score(y_true, y_pred):.4f}")

# Main execution
csv_file = '/content/SweetpredDB.csv'
dataset_rs = MultiTaskDataset(csv_file, mode='RS')
run_task('R/S Chirality', dataset_rs, 'node', 4)

dataset_ez = MultiTaskDataset(csv_file, mode='EZ')
run_task('E/Z Isomerism', dataset_ez, 'edge', 4)